In [1]:
from __future__ import (absolute_import, division,
                        print_function, unicode_literals)

import warnings
warnings.simplefilter('ignore')

import pandas as pd
import numpy as np
import os
import json
import time
import re

import psycopg2
import pandas.io.sql as sqlio

import scipy.stats as stats
import statsmodels.stats as smstats
import statsmodels.api as sm
from statsmodels.stats.multitest import multipletests

from multiprocessing import Process, Manager, Pool
import multiprocessing
from functools import partial

from collections import Counter

import seaborn as sns; sns.set()

import matplotlib
matplotlib.style.use('seaborn')
matplotlib.use('Agg')
import matplotlib.pyplot as plt
matplotlib.rcParams['backend'] = "Qt5Agg"
import matplotlib.ticker as ticker
from matplotlib.ticker import FuncFormatter

from IPython.display import display, Image

from adjustText import adjust_text

%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [4]:
# example of a configuration for connection to the "gtex_ipsa" PostgreSQL database. 
# Please update it according to your configuration settings

conn = psycopg2.connect(database='gtex_ipsa', user='postgres', host='localhost', port='5439')

# Import external script with necessary helper functions

In [ ]:
import importlib
import helper_functions as aaf
importlib.reload(aaf)

# Check RBP expression in RBP perturbation experiments

We check that RBP expression changed upon perturbation in the intended direction, i.e. decreased upon KD or KO and increased upon OE; otherwise, the experiment was excluded from the analysis.

In [277]:
sql = """SELECT * FROM experiment_metadata"""
experiment_metadata = pd.read_sql_query(sql, conn)
experiment_metadata['sample_number'] = experiment_metadata.apply(lambda x: len(x['list_of_ids'].split(' ')),1)
experiment_metadata = experiment_metadata.loc[experiment_metadata['experiment']!='no treat']

In [278]:
def get_gene_name(x):
    if x['experiment'][0]==' ':
        gene_name = x['experiment'].split(' ')[1]
    else:
        gene_name = x['experiment'].split(' ')[0]
    return gene_name
    
def get_perturbation(x):
    if x['experiment'][0]==' ':
        perturbation = x['experiment'].split(' ')[2]
    else:
        perturbation = x['experiment'].split(' ')[1]
    if perturbation.lower() in ['kd','sirna','shrna','ko','kd,']:
        perturbation = 'down'
    elif perturbation.lower()=='oe':
        perturbation = 'up'
    return perturbation
    
experiment_metadata['gene_name'] = experiment_metadata.apply(lambda x: get_gene_name(x),1)
experiment_metadata['perturbation'] = experiment_metadata.apply(lambda x: get_perturbation(x),1)

In [207]:
sql = """SELECT * FROM deseq"""
deseq = pd.read_sql_query(sql, conn)

In [279]:
experiment_metadata = pd.merge(experiment_metadata,deseq[['exper_id','gene_name','log2foldchange']],how='left',on=['exper_id','gene_name'])
experiment_metadata['log2foldchange'] = experiment_metadata['log2foldchange'].fillna(0)

In [280]:
def correct_direction(x):
    if x['perturbation']=='down':
        if x['log2foldchange']<0:
            return 'y'
        else:
            return 'n'
    elif x['perturbation']=='up':
        if x['log2foldchange']>0:
            return 'y'
        else:
            return 'n'
    else:
        return 'y'
experiment_metadata['correct_direction'] = experiment_metadata.apply(lambda x: correct_direction(x),1)

In [292]:
experiment_metadata = experiment_metadata.loc[(experiment_metadata['correct_direction']=='y')|(
    experiment_metadata['data_source']=='SRP041788')]
# SRP041788 stores NMD inactivation experiment in which we did not run DESeq2 analysis

In [347]:
len(experiment_metadata)
# including 419 RBP perturbations and 6 NMD inactivation experiments

425

In [348]:
experiment_metadata.to_csv('./experiment_metadata.tsv', 
                              sep=str('\t'),index=None,header=True,encoding='utf-8')

# Regulation of validated RBP-USE pairs

In [6]:
experiment_metadata = pd.read_csv('./experiment_metadata.tsv',delimiter="\t",index_col=None,header=0)

In [ ]:
# load the table with validated USEs

leus = pd.read_csv('./misc/Literature_examples_of_unproductive_splicing.txt',delimiter="\t",
                                   index_col=None,header=0)
leus = leus.fillna('')

In [150]:
cols = ['Target','AS event position', 'rMATS AS type','NMD site upstream hg19','NMD site downstream hg19',
       'NMD junction upstream hg19', 'NMD junction downstream hg19',
       'coding site upstream hg19', 'coding site downstream hg19',
       'coding junction upstream hg19', 'coding junction downstream hg19']
leus_df = leus.groupby(cols)['Regulators'].apply(';\n'.join).reset_index()
tmp = leus.groupby(cols)['Regulation'].apply(';\n'.join).reset_index()
leus_df = pd.merge(leus_df,tmp,how='inner',on=cols)
leus_df = leus_df.loc[leus_df['Target']!='']

In [152]:
def get_isoform_number(x):
    if 'NMD' in x['isoform']:
        return 1
    else:
        return 2

def get_average_expression(x):
    a = []
    for field in ['ijc_sample_1','sjc_sample_1','ijc_sample_2','sjc_sample_2']:
        a = a+x[field].split(',')
    return np.median(pd.Series(a).astype('float'))

def get_perturb_coef(x):
    if x['perturbation'] == 'up':
        perturb_coef = 1
    elif x['perturbation'] == 'down':
        perturb_coef = -1
    else:
        perturb_coef = 0
    return perturb_coef

def get_rmats_deseq(gene_name,input_df,AS_type):

    rMATS_coordinates = aaf.get_rMATS_coordinates(input_df,AS_type)
    if rMATS_coordinates is None:
        return None
    sql = aaf.get_sql_query_for_rmats(rMATS_coordinates)
    rmats = pd.read_sql_query(sql, conn)
    if len(rmats)==0:
        return None
    rmats = pd.merge(rmats,experiment_metadata[['exper_id','data_source','cell_lines','experiment','corr_coef','sample_number',
                                                'gene_name','perturbation']].rename(columns = {'gene_name':'regulator'}),
             how='inner',on='exper_id')
    adj_coef = rMATS_coordinates.loc[rMATS_coordinates['param']=='adj_coef']['val'].iloc[0]
    rmats['adj_coef'] = adj_coef
    rmats['delta_psi'] = adj_coef*rmats['corr_coef']*rmats['incleveldifference']
    rmats['median_local_expr'] = rmats.apply(lambda x: get_average_expression(x),1)

    sql = "SELECT * FROM deseq WHERE gene_name = '"+gene_name+"""'"""
    deseq = pd.read_sql_query(sql, conn)
    deseq.rename(columns = {'pvalue':'pvalue_deseq'},inplace=True)

    rmats_deseq = pd.merge(rmats.loc[(~rmats['experiment'].str.contains('|'.join(['CHX','UPF1'])))],deseq[['exper_id','log2foldchange','pvalue_deseq']],how='inner',on='exper_id')
    rmats_deseq.rename(columns = {'pvalue':'pvalue_rMATS'},inplace=True)

    # adjust the splicing and expression changes as if all the experiments had overexpressed RBPs
    rmats_deseq['perturb_coef'] = rmats_deseq.apply(lambda x: get_perturb_coef(x),1)
    rmats_deseq = rmats_deseq.loc[rmats_deseq['perturb_coef']!=0]
    rmats_deseq['delta_psi'] = rmats_deseq['perturb_coef']*rmats_deseq['delta_psi']
    rmats_deseq['log2foldchange'] = rmats_deseq['perturb_coef']*rmats_deseq['log2foldchange']

    return rmats_deseq

In [153]:
# we only look at USEs in which there are reported regulators and the AS class can be quantified with rMATS
test_leus_df = leus_df.loc[(leus_df['Regulators']!='')&(leus_df['rMATS AS type']!='complex')]

In [155]:
cols = pd.Series(leus_df.columns)
NMD_cols = list(cols.loc[cols.str.contains('NMD ')])
can_cols = list(cols.loc[cols.str.contains('coding ')])
          
a = []
r = []

for index, row in test_leus_df.iterrows():
    gene_name = row['Target']
    AS_event_position = row['AS event position']
    
    Regulators = pd.DataFrame([
        re.split(';\n|; \n',row['Regulators']),re.split(';\n|; \n',row['Regulation'])]).transpose()
    Regulators.columns = ['regulator','regulation']
    Regulators = Regulators.drop_duplicates('regulator')
    Regulators['regulator'] = Regulators['regulator'].str.replace('\n','')
    Regulators['regulation'] = Regulators['regulation'].str.replace('\n','')
    Regulators['regulation'] = Regulators['regulation'].str.replace(';','')
    Regulators['gene_name'] = gene_name
    Regulators['AS_event_position'] = AS_event_position
    r.append(Regulators)
    
    input_df = pd.DataFrame([list(row[NMD_cols+can_cols]),
                          list(NMD_cols+can_cols)]).transpose()
    input_df.columns=['long_id','isoform']
    input_df['number'] = input_df.apply(lambda x:get_isoform_number(x),1)
    input_df = input_df.loc[input_df['long_id']!='']
    AS_type = row['rMATS AS type']
    directory = './literature_events/'+gene_name+'_'+AS_event_position.replace("""'""",'prime')
    rmats_deseq = get_rmats_deseq(gene_name,input_df,AS_type)
    if rmats_deseq is not None and len(rmats_deseq)>0:
        rmats_deseq = rmats_deseq[['exper_id','data_source', 'cell_lines',
       'experiment', 'corr_coef','adj_coef','perturb_coef', 'sample_number', 'regulator', 'perturbation','median_local_expr','inclevel1','inclevel2',
       'delta_psi','pvalue_rMATS','log2foldchange', 'pvalue_deseq']]
        rmats_deseq['gene_name'] = gene_name
        rmats_deseq['AS_event_position'] = AS_event_position
        a.append(rmats_deseq)
        print('done: '+gene_name+' '+AS_event_position)
    else:
        print('no results: '+gene_name+' '+AS_event_position)

done: CPSF7 exon 4
done: DLG4 exon 17
done: EZH2 exon 8a
done: FUS exon 7
done: GABBR1 exon 15
done: HNRNPA2B1 3'UTR
done: HNRNPD 3'UTR
done: HNRNPDL 3'UTR
done: HNRNPL exon 6a
done: HNRNPLL exon 6a
done: HPS1 exon 18
done: LMNA exon 2a
done: MDM4 exon 6
done: PTBP1 exon 11
done: PTBP2 exon 10
done: PTBP3 exon 2
no results: RBFOX2 exon 5a
no results: RBFOX2 exon 6a
done: RBM10 exon 12
done: RBM10 exon 6
done: RBM39 exon 3
done: RBM5 exon 16
done: RBM5 exon 6
done: REST exon 4 ab
done: RPL10A intron 3
done: RPL3 exon 4
done: RPS3 exon 2
done: SF1 exon 3
done: SMNDC1 exon 3a
done: SNRNP70 exon 8a
done: SNRNP70 exon 8b
done: SNRNP70 exon 8c
done: SNRPB exon 3a
done: SRSF1 3'UTR
done: SRSF2 3'UTR
done: SRSF3 exon 4
done: SRSF4 exon 3a
done: SRSF5 exon 5a
done: SRSF5 intron 5
done: SRSF6 exon 3a
done: SRSF7 exon 3a
no results: SRSF8 3'UTR
done: SRSF9 exon 3a
done: TARDBP 3'UTR
done: TIA1 exon 7a
done: TRA2A exon 2a
done: TRA2B exon 2
done: U2AF1 exon 3b


In [157]:
# we obtained the combined rMATS output 
rmats_deseq_res = pd.concat(a)
rmats_deseq_res = rmats_deseq_res.reset_index(drop=True)

In [158]:
def get_psi_L(x):
    if x['corr_coef']*x['perturb_coef']==1:
        t = pd.Series(x['inclevel2'].split(','))
    else:
        t = pd.Series(x['inclevel1'].split(','))
    t = t.loc[t!='NA']
    t = t.astype('float')
    psi = np.mean(t)    
    if x['adj_coef']==-1:
        psi = 1-psi
    return psi

def get_psi_H(x):
    if x['corr_coef']*x['perturb_coef']==1:
        t = pd.Series(x['inclevel1'].split(','))
    else:
        t = pd.Series(x['inclevel2'].split(','))
    t = t.loc[t!='NA']
    t = t.astype('float')
    psi = np.mean(t)
    if x['adj_coef']==-1:
        psi = 1-psi
    return psi

rmats_deseq_res['psi_L'] = rmats_deseq_res.apply(lambda x: get_psi_L(x),1) # the value in the condition of low RBP expression
rmats_deseq_res['psi_H'] = rmats_deseq_res.apply(lambda x: get_psi_H(x),1) # the value in the condition of high RBP expression
rmats_deseq_res['psi_H_L'] = rmats_deseq_res['psi_H']-rmats_deseq_res['psi_L']

In [162]:
# we checked that psi_H_L and delta_psi values are equal.

rmats_deseq_res['temp'] = rmats_deseq_res['psi_H_L']-rmats_deseq_res['delta_psi']
rmats_deseq_res['temp'].min(),rmats_deseq_res['temp'].max()

(-0.0005000000000000976, 0.0005000000000001115)

In [163]:
rmats_deseq_res['delta_psi'] = rmats_deseq_res['psi_H_L']
rmats_deseq_res = rmats_deseq_res.drop('psi_H_L',1)

In [164]:
# get psi in control conditions
def get_psi_ctl(x):
    if x['perturbation']=='down':
        return x['psi_H']
    else:
        return x['psi_L']

# get psi in experimental conditions
def get_psi_exp(x):
    if x['perturbation']=='down':
        return x['psi_L']
    else:
        return x['psi_H']

rmats_deseq_res['psi_ctl'] = rmats_deseq_res.apply(lambda x:get_psi_ctl(x),1)
rmats_deseq_res['psi_exp'] = rmats_deseq_res.apply(lambda x:get_psi_exp(x),1)

In [270]:
def get_category(x):
    if x['regulator']==x['gene_name']:
        category = 'auto'
    elif x['regulation']=='NMD-inhibiting':
        category = 'cross, NMD-inhibiting'
    elif x['regulation']=='NMD-promoting':
        category = 'cross, NMD-promoting'
    else:
        category = ''
    return category

Regulators_res = pd.concat(r)
Regulators_res= Regulators_res.reset_index(drop=True)
Regulators_res = Regulators_res.loc[Regulators_res['regulator']!='']
Regulators_res['category'] = Regulators_res.apply(lambda x: get_category(x),1)

experiment_metadata['t']=1
tmp1 = experiment_metadata.loc[experiment_metadata['perturbation']=='up'].groupby(['gene_name']).agg({'t':sum}).reset_index().rename(columns ={'gene_name':'regulator','t':'t_up'})
tmp1_1 = experiment_metadata.loc[experiment_metadata['perturbation']=='down'].groupby(['gene_name']).agg({'t':sum}).reset_index().rename(columns ={'gene_name':'regulator','t':'t_down'})

tmp1 = pd.merge(tmp1,tmp1_1,how='outer',on='regulator')
tmp1[['t_up','t_down']] = tmp1[['t_up','t_down']].fillna(0).astype('int')

tmp2 = Regulators_res[['regulator']].drop_duplicates()
tmp2 = pd.merge(tmp2,tmp1,how='left',on='regulator')
tmp2[['t_up','t_down']] = tmp2[['t_up','t_down']].fillna(0).astype('int')

Regulators_res = pd.merge(Regulators_res,tmp2,how='left',on='regulator')

Regulators_res[['t_up','t_down']] = Regulators_res[['t_up','t_down']].fillna(0).astype('int')
Regulators_res['n'] = ((Regulators_res['t_up']>0)|(Regulators_res['t_down']>0)).astype('int')
Regulators_res['n_up'] = (Regulators_res['t_up']>0).astype('int')
Regulators_res['n_down'] = (Regulators_res['t_down']>0).astype('int')

In [290]:
# We convert the results of differential splicing (DS) and differential expression (DE) to scores (ranks).

def get_rank_DS(x,abs_delta_psi_thr):    
    r_DS = 0
    thr = abs_delta_psi_thr
    
    if abs(x['delta_psi'])>thr:
        r_DS = 1
        if x['fdr_rMATS']<0.05:
            r_DS = 1.5
    if x['delta_psi']<0:
        r_DS = r_DS*(-1)
    return r_DS

def get_rank_DE(x,abs_log2foldchange_thr):
    r_DE = 0
    thr = abs_log2foldchange_thr
    if abs(x['log2foldchange'])>thr:
        r_DE = 1
        if x['fdr_deseq']<0.05:
            r_DE = 1.5
    if x['log2foldchange']<0:
        r_DE = r_DE*(-1)
    return r_DE

rank_DS_thr = 0
rank_DE_thr = 0

thr_local_expression = 15
abs_log2foldchange_thr = 0
abs_delta_psi_thr = 0.1

rmats_deseq_t = rmats_deseq_res.copy()
targets = rmats_deseq_t[['gene_name','AS_event_position']].drop_duplicates()
a = []
for index,row in targets.iterrows():
    tmp = rmats_deseq_t.loc[(rmats_deseq_t['gene_name']==row['gene_name'])&(
        rmats_deseq_t['AS_event_position']==row['AS_event_position'])]
    tmp['fdr_rMATS'] = multipletests(tmp['pvalue_rMATS'],method='fdr_bh')[1]
    tmp['fdr_deseq'] = multipletests(tmp['pvalue_deseq'],method='fdr_bh')[1]
    a.append(tmp)
rmats_deseq_t = pd.concat(a)
rmats_deseq_t['rank_DS'] = rmats_deseq_t.apply(lambda x: get_rank_DS(x,abs_delta_psi_thr),1)
rmats_deseq_t['rank_DE'] = rmats_deseq_t.apply(lambda x: get_rank_DE(x,abs_log2foldchange_thr),1)
rmats_deseq_t['rank_DS'] = rmats_deseq_t['rank_DS']*(rmats_deseq_t['median_local_expr']>thr_local_expression).astype('int')
rmats_deseq_t['rank_DE'] = rmats_deseq_t['rank_DE']*(rmats_deseq_t['median_local_expr']>thr_local_expression).astype('int')

rmats_deseq_t = pd.merge(Regulators_res.loc[Regulators_res['n']==1][['regulator','gene_name','regulation','AS_event_position','category']].drop_duplicates(),
                         rmats_deseq_t, how='left', on = ['gene_name','AS_event_position','regulator'])

res = rmats_deseq_t.groupby(['gene_name','AS_event_position','regulator','regulation','category']).agg({'rank_DS':mean,'rank_DE':mean,'psi_L':mean,'psi_H':mean,'psi_ctl':mean,'psi_exp':mean}).reset_index()

# We test if the observed changes in splicing match the expectations from literature
res['co_DS'] = ((res['regulation']=='NMD-promoting').astype('int')*2-1)*res['rank_DS']
res['anti_DS'] = (res['co_DS']<-abs(rank_DS_thr)).astype('int')
res['co_DS'] = (res['co_DS']>abs(rank_DS_thr)).astype('int')

# We test if the observed changes in expression match the expectations from literature
res['co_DE'] = ((res['regulation']=='NMD-promoting').astype('int')*2-1)*res['rank_DE']*(-1)
res['anti_DE'] = (res['co_DE']<-abs(rank_DE_thr)).astype('int')
res['co_DE'] = (res['co_DE']>abs(rank_DE_thr)).astype('int')

def get_co(x):
    if x['regulator']==x['gene_name']:
        if x['co_DS']==1:
            return 1
        else:
            return 0
    else:
        if x['co_DS']==1 and x['co_DE']==1:
            return 1
        else:
            return 0

def get_anti(x):
    if x['regulator']==x['gene_name']:
        if x['anti_DS']==1:
            return 1
        else:
            return 0
    else:
        if x['anti_DS']==1 and x['anti_DE']==1:
            return 1
        else:
            return 0
        
res['co'] = res.apply(lambda x:get_co(x),1) # both DS and DE changes match the expectations
res['anti'] = res.apply(lambda x:get_anti(x),1) # both DS and DE changes are opposite to expected
res['n']=1

In [306]:
rmats_deseq_t.to_csv('./misc/perturbation_results_details.tsv', 
                              sep=str('\t'),index=None,header=True,encoding='utf-8')
res.to_csv('./misc/perturbation_results_grouped.tsv', 
                              sep=str('\t'),index=None,header=True,encoding='utf-8')

# Identification of annotated USEs

In [ ]:
# we extract "exon" and "stop codon" objects from the annotation using the following bash commands:
# awk '$3 == "exon"' gencode.v35lift37.annotation.gtf | gzip > hg19_exons.gtf.gz
# awk '$3 == "stop_codon"' gencode.v35lift37.annotation.gtf | gzip > hg19_stop_codons.gtf.gz

## Extraction of exon borders and stop codon positions

In [57]:
exons = pd.read_csv('./Annotation/hg19_exons.gtf.gz',delimiter="\t",index_col=None,header=None,compression='gzip')
exons.columns = ['chr','tmp1','tmp0','start','end','tmp2','str','tmp3','info']
exons.drop(['tmp1','tmp2','tmp3','tmp0'],1,inplace=True)
exons['exon'] = exons['chr']+'_'+exons['start'].astype('str')+'_'+exons['end'].astype('str')+'_'+exons['str']
exons.drop(['start','end','str'],1,inplace=True)

stop_codons = pd.read_csv('./Annotation/hg19_stop_codons.gtf.gz',delimiter="\t",index_col=None,header=None,compression='gzip')
stop_codons.columns = ['chr','tmp1','tmp0','start','end','tmp2','str','tmp3','info']
stop_codons.drop(['tmp1','tmp2','tmp3','tmp0'],1,inplace=True)

exons['transcript_id'] = exons['info'].str.split(';',expand=True)[1].str.split('"',expand=True)[1]
exons['gene_name'] = exons['info'].str.split(';',expand=True)[3].str.split('"',expand=True)[1]
exons['exon_number'] = exons['info'].str.split(';',expand=True)[6].str.split(' ',expand=True)[2]
exons['transcript_type'] = exons['info'].str.split(';',expand=True)[4].str.split('"',expand=True)[1]

exons['ccds'] = exons['info'].str.contains('ccds').astype('int')

stop_codons['transcript_id'] = stop_codons['info'].str.split(';',expand=True)[1].str.split('"',expand=True)[1]

# we do not consider transcripts withous stop codons
exons = pd.merge(exons[['exon','gene_name','transcript_id','transcript_type','ccds','exon_number']],
stop_codons.rename(columns = {'start':'stop_start','end':'stop_end'})[['transcript_id','stop_start','stop_end']],
how='inner',on='transcript_id')

exons['exon_start'] = exons['exon'].str.split('_',expand=True)[1].astype('int')
exons['exon_end'] = exons['exon'].str.split('_',expand=True)[2].astype('int')

exons = exons.loc[exons['exon_number'].str.isnumeric()]
exons['exon_number'] = exons['exon_number'].astype('int')

gr = exons.groupby('transcript_id').agg({'exon_number':max}).reset_index()

exons = pd.merge(exons,gr.rename(columns={'exon_number':'max_exon_number'}),how='left',on='transcript_id')

exons.sort_values(['transcript_id','exon_start'],ascending=[True,True],inplace=True)

exons = exons.drop_duplicates(['exon','transcript_id']).reset_index(drop=True)

transcript_id_pred = ''
exon_end_pred = 0
upst_junctions = []
for a in exons[['transcript_id','exon_start','exon_end']].values:
    if (exon_end_pred>0) and (a[0]==transcript_id_pred):
        upst_junction = str(exon_end_pred)+'_'+str(a[1])
    else:
        upst_junction = ''
    upst_junctions.append(upst_junction)
    exon_end_pred = a[2]
    transcript_id_pred = a[0]
exons['upst_junction'] = upst_junctions

exons.sort_values(['transcript_id','exon_start'],ascending=[True,False],inplace=True)

transcript_id_pred = ''
exon_start_pred = 0
downst_junctions = []
for a in exons[['transcript_id','exon_start','exon_end']].values:
    if (exon_start_pred>0) and (a[0]==transcript_id_pred):
        downst_junction = str(a[2])+'_'+str(exon_start_pred)
    else:
        downst_junction = ''
    downst_junctions.append(downst_junction)
    exon_start_pred = a[1]
    transcript_id_pred = a[0]
exons['downst_junction'] = downst_junctions

exons = exons.sort_values(['transcript_id','exon_start'],ascending=[True,True]).reset_index(drop=True)

## Poison exons

In [7]:
# We identify exons that are included in at least one NMD transcript but skipped in all protein-coding transcripts
# We check the following conditions: 1) the presence of NMD isoform with inclusion 2) the presence of coding isoform with skipping 
# 3) absence of coding isoforms with inclusion 4) absence of NMD isoforms with skipping
CE_in_nmd_trs = exons.loc[(exons['transcript_type']=='nonsense_mediated_decay')&(
exons['max_exon_number']>exons['exon_number'])&(exons['exon_number']>1)]
CE_in_nmd_trs['skip_junction'] = CE_in_nmd_trs['upst_junction'].str.split('_',expand=True)[0]+'_'+CE_in_nmd_trs['downst_junction'].str.split('_',expand=True)[1]

exons_in_coding_trs = exons.loc[(exons['transcript_type']=='protein_coding')]
exons_in_coding_trs = pd.concat([exons_in_coding_trs[['upst_junction','gene_name']].drop_duplicates().rename(columns = {'upst_junction':'skip_junction'}),
exons_in_coding_trs[['downst_junction','gene_name']].drop_duplicates().rename(columns = {'downst_junction':'skip_junction'})]).drop_duplicates().reset_index(drop=True)
exons_in_coding_trs = exons_in_coding_trs.loc[exons_in_coding_trs['skip_junction']!='']
exons_in_coding_trs['skip_coding_exists'] = 1

CE_in_nmd_trs = pd.merge(CE_in_nmd_trs,exons_in_coding_trs,how='left',on=['skip_junction','gene_name'])
CE_in_nmd_trs['skip_coding_exists'] = CE_in_nmd_trs['skip_coding_exists'].fillna(0).astype('int')

exons_in_coding_trs.rename(columns={'skip_coding_exists':'upst_coding_exists','skip_junction':'upst_junction'},inplace=True)
CE_in_nmd_trs = pd.merge(CE_in_nmd_trs,exons_in_coding_trs,how='left',on=['upst_junction','gene_name'])
CE_in_nmd_trs['upst_coding_exists'] = CE_in_nmd_trs['upst_coding_exists'].fillna(0).astype('int')

exons_in_coding_trs.rename(columns={'upst_coding_exists':'downst_coding_exists','upst_junction':'downst_junction'},inplace=True)
CE_in_nmd_trs = pd.merge(CE_in_nmd_trs,exons_in_coding_trs,how='left',on=['downst_junction','gene_name'])
CE_in_nmd_trs['downst_coding_exists'] = CE_in_nmd_trs['downst_coding_exists'].fillna(0).astype('int')

exons_in_nmd_trs = exons.loc[(exons['transcript_type']=='nonsense_mediated_decay')]
exons_in_nmd_trs = pd.concat([exons_in_nmd_trs[['upst_junction','gene_name']].drop_duplicates().rename(columns = {'upst_junction':'skip_junction'}),
exons_in_nmd_trs[['downst_junction','gene_name']].drop_duplicates().rename(columns = {'downst_junction':'skip_junction'})]).drop_duplicates().reset_index(drop=True)
exons_in_nmd_trs = exons_in_nmd_trs.loc[exons_in_nmd_trs['skip_junction']!='']
exons_in_nmd_trs['skip_nmd_exists'] = 1
CE_in_nmd_trs = pd.merge(CE_in_nmd_trs,exons_in_nmd_trs,how='left',on=['skip_junction','gene_name'])
CE_in_nmd_trs['skip_nmd_exists'] = CE_in_nmd_trs['skip_nmd_exists'].fillna(0).astype('int')

CE_in_nmd_trs['str']=CE_in_nmd_trs['exon'].str.split('_',expand=True)[3]

def get_stop_rel_position(x):
    stop_rel_position = ''
    if (x['exon_start']<=x['stop_start'] and x['stop_start']<=x['exon_end']) or (x['exon_start']<=x['stop_end'] and x['stop_end']<=x['exon_end']):
        stop_rel_position = 'stop_within'
    elif x['str']=='+':
        if x['stop_start']>x['exon_end']:
            stop_rel_position = 'stop_downst'
        elif x['stop_end']<x['exon_start']:
            stop_rel_position = 'stop_upst'
    elif x['str']=='-':
        if x['stop_start']>x['exon_end']:
            stop_rel_position = 'stop_upst'
        elif x['stop_end']<x['exon_start']:
            stop_rel_position = 'stop_downst'
    return stop_rel_position

CE_in_nmd_trs['inc_specific_to_NMD_transcript'] = ((CE_in_nmd_trs['skip_coding_exists']==1)&(CE_in_nmd_trs['skip_nmd_exists']==0)&(CE_in_nmd_trs['upst_coding_exists']==0)&(CE_in_nmd_trs['downst_coding_exists']==0)).astype('int')
CE_in_nmd_trs['stop_rel_position'] = CE_in_nmd_trs.apply(lambda x:get_stop_rel_position(x),1)
poison_exons = CE_in_nmd_trs.loc[CE_in_nmd_trs['inc_specific_to_NMD_transcript']==1]
poison_exons['chr'] = poison_exons['exon'].str.split('_',expand=True)[0]
poison_exons['str'] = poison_exons['exon'].str.split('_',expand=True)[3]
poison_exons = poison_exons[['chr','str','gene_name','transcript_id',
'upst_junction','downst_junction','skip_junction','stop_rel_position']].drop_duplicates().reset_index(drop=True)

In [8]:
Counter(poison_exons['stop_rel_position'])

Counter({'stop_within': 1753, 'stop_downst': 352, 'stop_upst': 207})

In [58]:
CE_in_coding_trs = exons.loc[(exons['transcript_type']=='protein_coding')&(
exons['max_exon_number']>exons['exon_number'])&(exons['exon_number']>1)]
CE_in_coding_trs['skip_junction'] = CE_in_coding_trs['upst_junction'].str.split('_',expand=True)[0]+'_'+CE_in_coding_trs['downst_junction'].str.split('_',expand=True)[1]

exons_in_coding_trs = exons.loc[(exons['transcript_type']=='protein_coding')]
exons_in_coding_trs = pd.concat([exons_in_coding_trs[['upst_junction','gene_name']].drop_duplicates().rename(columns = {'upst_junction':'skip_junction'}),
exons_in_coding_trs[['downst_junction','gene_name']].drop_duplicates().rename(columns = {'downst_junction':'skip_junction'})]).drop_duplicates().reset_index(drop=True)
exons_in_coding_trs = exons_in_coding_trs.loc[exons_in_coding_trs['skip_junction']!='']
exons_in_coding_trs['skip_coding_exists'] = 1

CE_in_coding_trs = pd.merge(CE_in_coding_trs,exons_in_coding_trs,how='left',on=['skip_junction','gene_name'])
CE_in_coding_trs['skip_coding_exists'] = CE_in_coding_trs['skip_coding_exists'].fillna(0).astype('int')

## Essential exons

In [20]:
# We idenify all exons that are skipped in at least one NMD transcript but are included in all protein-coding transcripts
CE_in_coding_trs = exons.loc[(exons['transcript_type']=='protein_coding')&(
exons['max_exon_number']>exons['exon_number'])&(exons['exon_number']>1)]
CE_in_coding_trs['skip_junction'] = CE_in_coding_trs['upst_junction'].str.split('_',expand=True)[0]+'_'+CE_in_coding_trs['downst_junction'].str.split('_',expand=True)[1]

exons_in_coding_trs = exons.loc[(exons['transcript_type']=='protein_coding')]
exons_in_coding_trs = pd.concat([exons_in_coding_trs[['upst_junction','gene_name']].drop_duplicates().rename(columns = {'upst_junction':'skip_junction'}),
exons_in_coding_trs[['downst_junction','gene_name']].drop_duplicates().rename(columns = {'downst_junction':'skip_junction'})]).drop_duplicates().reset_index(drop=True)
exons_in_coding_trs = exons_in_coding_trs.loc[exons_in_coding_trs['skip_junction']!='']
exons_in_coding_trs['skip_coding_exists'] = 1

CE_in_coding_trs = pd.merge(CE_in_coding_trs,exons_in_coding_trs,how='left',on=['skip_junction','gene_name'])
CE_in_coding_trs['skip_coding_exists'] = CE_in_coding_trs['skip_coding_exists'].fillna(0).astype('int')

CE_in_coding_trs = CE_in_coding_trs.loc[CE_in_coding_trs['skip_coding_exists']==0][
    ['gene_name',
     'upst_junction',
     'downst_junction',
     'skip_junction']].drop_duplicates()

CE_in_nmd_trs = exons.loc[(exons['transcript_type']=='nonsense_mediated_decay')&(exons['upst_junction']!='')&(exons['downst_junction']!='')]
CE_in_nmd_trs = CE_in_nmd_trs[['gene_name','upst_junction','downst_junction']].drop_duplicates()
CE_in_nmd_trs['inc_nmd_exists'] = 1
CE_in_coding_trs = pd.merge(CE_in_coding_trs,CE_in_nmd_trs,how='left',on=['gene_name','upst_junction','downst_junction'])
CE_in_coding_trs['inc_nmd_exists'] = CE_in_coding_trs['inc_nmd_exists'].fillna(0).astype('int')
CE_in_coding_trs = CE_in_coding_trs.loc[CE_in_coding_trs['inc_nmd_exists']==0][
    ['gene_name',
     'upst_junction',
     'downst_junction',
     'skip_junction']].drop_duplicates()

CE_in_nmd_trs = exons.loc[(exons['transcript_type']=='nonsense_mediated_decay')&(exons['upst_junction']!='')]
CE_in_nmd_trs.rename(columns = {'upst_junction':'skip_junction'},inplace=True)
CE_in_nmd_trs['chr'] = CE_in_nmd_trs['exon'].str.split('_',expand=True)[0]
CE_in_nmd_trs['str'] = CE_in_nmd_trs['exon'].str.split('_',expand=True)[3]
CE_in_nmd_trs = CE_in_nmd_trs[['gene_name','chr','str','transcript_id','stop_start','stop_end','skip_junction','exon_start','exon_end']].drop_duplicates()
CE_in_nmd_trs.rename(columns={'exon_start':'downst_exon_start','exon_end':'downst_exon_end'},inplace=True)

temp_nmd = exons.loc[(exons['transcript_type']=='nonsense_mediated_decay')&(exons['downst_junction']!='')]
temp_nmd.rename(columns = {'downst_junction':'skip_junction'},inplace=True)
temp_nmd = temp_nmd[['gene_name','transcript_id','skip_junction','exon_start','exon_end']].drop_duplicates()
temp_nmd.rename(columns={'exon_start':'upst_exon_start','exon_end':'upst_exon_end'},inplace=True)

CE_in_nmd_trs = pd.merge(CE_in_nmd_trs,temp_nmd,how='inner',on=['gene_name','transcript_id','skip_junction'])
CE_in_nmd_trs = pd.merge(CE_in_nmd_trs,CE_in_coding_trs,how='inner',on=['gene_name','skip_junction'])
CE_in_nmd_trs.rename(columns= {'upst_junction':'upst_junction_cod',
                               'downst_junction':'downst_junction_cod',
                              'upst_exon_end':'skip_junction_start',
                              'downst_exon_start':'skip_junction_end'},inplace=True)
CE_in_nmd_trs['stop_len'] = CE_in_nmd_trs['stop_end']-CE_in_nmd_trs['stop_start']+1
CE_in_nmd_trs.reset_index(drop=True,inplace=True)

def get_stop_rel_position_ess(x):
    stop_rel_position = ''
    if (x['stop_len']<3) and (x['stop_start']<=x['skip_junction_start'] and (x['stop_start']+2)>=x['skip_junction_start']):
        stop_rel_position = 'stop_within'
    elif x['str']=='+':
        if x['stop_start']>=x['skip_junction_end']:
            stop_rel_position = 'stop_downst'
        elif x['stop_end']<=x['skip_junction_start']:
            stop_rel_position = 'stop_upst'          
    elif x['str']=='-':
        if x['stop_start']>=x['skip_junction_end']:
            stop_rel_position = 'stop_upst'
        elif x['stop_end']<=x['skip_junction_start']:
            stop_rel_position = 'stop_downst'
    return stop_rel_position

CE_in_nmd_trs['stop_rel_position'] = CE_in_nmd_trs.apply(lambda x:get_stop_rel_position_ess(x),1)
essential_exons = CE_in_nmd_trs.copy()
essential_exons = essential_exons[['chr','str','gene_name','transcript_id',
                                   'upst_junction_cod',
                                   'downst_junction_cod',
                                   'skip_junction',
                                   'stop_rel_position']].drop_duplicates().reset_index(drop=True).rename(columns={'upst_junction_cod':'upst_junction','downst_junction_cod':'downst_junction'})

In [21]:
Counter(essential_exons['stop_rel_position'])

Counter({'stop_upst': 256, 'stop_downst': 1953, 'stop_within': 33})

## Control CE

In [23]:
# We identify exons that are included and skipped in different protein-coding transcripts
# excluding essential and poison exons
CE_in_coding_trs = exons.loc[(exons['transcript_type']=='protein_coding')&(
exons['max_exon_number']>exons['exon_number'])&(exons['exon_number']>1)]
CE_in_coding_trs['skip_junction'] = CE_in_coding_trs['upst_junction'].str.split('_',expand=True)[0]+'_'+CE_in_coding_trs['downst_junction'].str.split('_',expand=True)[1]

exons_in_coding_trs = exons.loc[(exons['transcript_type']=='protein_coding')]
exons_in_coding_trs = pd.concat([exons_in_coding_trs[['upst_junction','gene_name']].drop_duplicates().rename(columns = {'upst_junction':'skip_junction'}),
exons_in_coding_trs[['downst_junction','gene_name']].drop_duplicates().rename(columns = {'downst_junction':'skip_junction'})]).drop_duplicates().reset_index(drop=True)
exons_in_coding_trs = exons_in_coding_trs.loc[exons_in_coding_trs['skip_junction']!='']
exons_in_coding_trs['skip_coding_exists'] = 1

CE_in_coding_trs = pd.merge(CE_in_coding_trs,exons_in_coding_trs,how='left',on=['skip_junction','gene_name'])
CE_in_coding_trs['skip_coding_exists'] = CE_in_coding_trs['skip_coding_exists'].fillna(0).astype('int')

CE_in_coding_trs = CE_in_coding_trs.loc[CE_in_coding_trs['skip_coding_exists']==1]

CE_in_coding_trs['chr'] = CE_in_coding_trs['exon'].str.split('_',expand=True)[0]
CE_in_coding_trs['str'] = CE_in_coding_trs['exon'].str.split('_',expand=True)[3]

essential_exons['essential_exon']=1
CE_in_coding_trs = pd.merge(CE_in_coding_trs,
                            essential_exons[['chr', 'str', 'gene_name', 'upst_junction', 
                                             'downst_junction','skip_junction','essential_exon']].drop_duplicates(),
how='left',on=['chr', 'str', 'gene_name', 'upst_junction', 'downst_junction','skip_junction'])
essential_exons.drop('essential_exon',1,inplace=True)

poison_exons['poison_exon']=1
CE_in_coding_trs = pd.merge(CE_in_coding_trs,
                            poison_exons[['chr', 'str', 'gene_name', 'upst_junction', 
                                             'downst_junction','skip_junction','poison_exon']].drop_duplicates(),
how='left',on=['chr', 'str', 'gene_name', 'upst_junction', 'downst_junction','skip_junction'])
poison_exons.drop('poison_exon',1,inplace=True)

CE_in_coding_trs[['poison_exon','essential_exon']]=CE_in_coding_trs[['poison_exon','essential_exon']].fillna(0).astype('int') 
CE_in_coding_trs = CE_in_coding_trs.loc[(CE_in_coding_trs['poison_exon']==0)&(CE_in_coding_trs['essential_exon']==0)]

def get_stop_rel_position(x):
    stop_rel_position = ''
    if (x['exon_start']<=x['stop_start'] and x['stop_start']<=x['exon_end']) or (x['exon_start']<=x['stop_end'] and x['stop_end']<=x['exon_end']):
        stop_rel_position = 'stop_within_CE'
    elif x['str']=='+':
        if x['stop_start']>x['exon_end']:
            stop_rel_position = 'stop_downst'
        elif x['stop_end']<x['exon_start']:
            stop_rel_position = 'stop_upst'
    elif x['str']=='-':
        if x['stop_start']>x['exon_end']:
            stop_rel_position = 'stop_upst'
        elif x['stop_end']<x['exon_start']:
            stop_rel_position = 'stop_downst'
    return stop_rel_position

CE_in_coding_trs['stop_rel_position'] = CE_in_coding_trs.apply(lambda x:get_stop_rel_position(x),1)

coding_ce = CE_in_coding_trs.loc[CE_in_coding_trs['stop_rel_position']=='stop_downst'][['chr','str','gene_name',
                                   'upst_junction',
                                   'downst_junction',
                                   'skip_junction','stop_rel_position']].drop_duplicates().reset_index(drop=True)

In [24]:
len(coding_ce)

13292

In [26]:
coding_ce.to_csv('./misc/hg19_cassette_exons_coding.tsv', sep=str('\t'),header=True,index=None)

## Alternative 5' and 3' splice sites

In [31]:
# we consider alternative ends of junctions regardeless of the strand 
# (so, disregarding the classification as 5' or 3' at this step)
sites_in_nmd_trs = exons.loc[(exons['transcript_type']=='nonsense_mediated_decay')&(exons['downst_junction']!='')]
temp_nmd_trs = exons.loc[(exons['transcript_type']=='nonsense_mediated_decay')&(exons['upst_junction']!='')]
sites_in_nmd_trs = pd.merge(sites_in_nmd_trs,
                            temp_nmd_trs[['gene_name','transcript_id',
                                          'exon_end','upst_junction']].rename(columns={'upst_junction':'downst_junction',
                                                                                                        'exon_end':'flank_exon_end'}),
how='inner',on=['gene_name','transcript_id','downst_junction'])

sites_in_nmd_trs = sites_in_nmd_trs[['exon','gene_name','transcript_id','stop_start',
                  'stop_end','exon_start','exon_end',
                  'downst_junction','flank_exon_end']].drop_duplicates().reset_index(drop=True)
sites_in_nmd_trs['flank_exon_start'] = sites_in_nmd_trs['downst_junction'].str.split('_',expand=True)[1].astype('int')
sites_in_nmd_trs['flank_exon_end']  = sites_in_nmd_trs['flank_exon_end'].astype('int')
sites_in_nmd_trs = sites_in_nmd_trs.drop(['downst_junction'],1).drop_duplicates().reset_index(drop=True)

sites_in_coding_trs = exons.loc[(exons['transcript_type']=='protein_coding')&(exons['downst_junction']!='')]
temp_coding_trs = exons.loc[(exons['transcript_type']=='protein_coding')&(exons['upst_junction']!='')]
sites_in_coding_trs = pd.merge(sites_in_coding_trs,
                            temp_coding_trs[['gene_name','transcript_id',
                                          'exon_end','upst_junction']].rename(columns={'upst_junction':'downst_junction',
                                                                                    'exon_end':'flank_exon_end'}),
how='inner',on=['gene_name','transcript_id','downst_junction'])

sites_in_coding_trs = sites_in_coding_trs[['gene_name',
                                           'exon_start','exon_end',
                                          'downst_junction','flank_exon_end']].drop_duplicates().reset_index(drop=True)
sites_in_coding_trs['flank_exon_start'] = sites_in_coding_trs['downst_junction'].str.split('_',expand=True)[1].astype('int')
sites_in_coding_trs['flank_exon_end']  = sites_in_coding_trs['flank_exon_end'].astype('int')
sites_in_coding_trs = sites_in_coding_trs.drop(['downst_junction'],1).drop_duplicates().reset_index(drop=True)
sites_in_coding_trs['coding_site_exists']=1

sites_in_nmd_trs = pd.merge(sites_in_nmd_trs,
                            sites_in_coding_trs[['gene_name','exon_start','exon_end',
                                      'flank_exon_start','flank_exon_end','coding_site_exists']].drop_duplicates(),
how='left',
on=['gene_name','exon_start','exon_end','flank_exon_start','flank_exon_end'])
sites_in_nmd_trs['coding_site_exists'] = sites_in_nmd_trs['coding_site_exists'].fillna(0).astype('int')

sites_in_nmd_trs = sites_in_nmd_trs.loc[sites_in_nmd_trs['coding_site_exists']==0].drop('coding_site_exists',1)

sites_in_nmd_trs = pd.merge(sites_in_nmd_trs,
                            sites_in_coding_trs[['gene_name','exon_start','exon_end',
                    'flank_exon_start','flank_exon_end']].drop_duplicates().rename(columns={'exon_end':'exon_end_coding'}),
how='left',
on=['gene_name','exon_start','flank_exon_start','flank_exon_end'])

sites_in_nmd_trs = sites_in_nmd_trs.loc[~sites_in_nmd_trs['exon_end_coding'].isna()]
sites_in_nmd_trs['exon_end_coding'] = sites_in_nmd_trs['exon_end_coding'].astype('int')
sites_in_nmd_trs = sites_in_nmd_trs.loc[sites_in_nmd_trs['exon_end_coding']!=sites_in_nmd_trs['exon_end']]


sites_in_nmd_trs_2 = exons.loc[(exons['transcript_type']=='nonsense_mediated_decay')&(exons['downst_junction']!='')]
temp_nmd_trs = exons.loc[(exons['transcript_type']=='nonsense_mediated_decay')&(exons['upst_junction']!='')]
sites_in_nmd_trs_2 = pd.merge(sites_in_nmd_trs_2,
                            temp_nmd_trs[['gene_name','transcript_id',
                                          'exon_end','upst_junction']].rename(columns={'upst_junction':'downst_junction',
                                                                                                        'exon_end':'flank_exon_end'}),
how='inner',on=['gene_name','transcript_id','downst_junction'])
sites_in_nmd_trs_2 = sites_in_nmd_trs_2[['gene_name',
                                         'exon_start','exon_end',
                              'downst_junction','flank_exon_end']].drop_duplicates().reset_index(drop=True)
sites_in_nmd_trs_2['flank_exon_start'] = sites_in_nmd_trs_2['downst_junction'].str.split('_',expand=True)[1].astype('int')
sites_in_nmd_trs_2['flank_exon_end']  = sites_in_nmd_trs_2['flank_exon_end'].astype('int')
sites_in_nmd_trs_2 = sites_in_nmd_trs_2.drop(['downst_junction'],1).drop_duplicates().reset_index(drop=True)
sites_in_nmd_trs_2.rename(columns={'exon_end':'exon_end_coding'},inplace=True)
sites_in_nmd_trs_2['alt_nmd_site_exists'] = 1

sites_in_nmd_trs = pd.merge(sites_in_nmd_trs,sites_in_nmd_trs_2[['gene_name','exon_end_coding','flank_exon_start','alt_nmd_site_exists']].drop_duplicates(),
how='left',on=['gene_name','exon_end_coding','flank_exon_start'])
sites_in_nmd_trs['alt_nmd_site_exists'] = sites_in_nmd_trs['alt_nmd_site_exists'].fillna(0).astype('int')
sites_in_nmd_trs = sites_in_nmd_trs.loc[sites_in_nmd_trs['alt_nmd_site_exists']==0].drop('alt_nmd_site_exists',1)

sites_in_nmd_trs[['stop_start', 'stop_end',
       'exon_start', 'exon_end', 'flank_exon_end', 'flank_exon_start',
       'exon_end_coding']] = sites_in_nmd_trs[['stop_start', 'stop_end',
       'exon_start', 'exon_end', 'flank_exon_end', 'flank_exon_start',
       'exon_end_coding']].astype('int')

sites_in_nmd_trs['str'] = sites_in_nmd_trs['exon'].str.split('_',expand=True)[3]

def get_stop_rel_position_site(x):
    stop_rel_position = ''
    min_end = min([x['exon_end_coding'],x['exon_end']])
    max_end = max([x['exon_end_coding'],x['exon_end']])
    if (x['exon_end_coding']<x['exon_end']) and ((x['exon_end_coding']<=x['stop_start'] and x['stop_start']<=x['exon_end']) or (x['exon_end_coding']<=x['stop_end'] and x['stop_end']<=x['exon_end'])):
        stop_rel_position = 'stop_within'
    elif x['str']=='+':
        if x['stop_start']>=x['flank_exon_start']:
            stop_rel_position = 'stop_downst'
        elif x['stop_end']<=min_end:
            stop_rel_position = 'stop_upst'
    elif x['str']=='-':
        if x['stop_start']>=x['flank_exon_start']:
            stop_rel_position = 'stop_upst'
        elif x['stop_end']<=min_end:
            stop_rel_position = 'stop_downst'
    return stop_rel_position

sites_in_nmd_trs['stop_rel_position'] = sites_in_nmd_trs.apply(lambda x:get_stop_rel_position_site(x),1)
alt_ends = sites_in_nmd_trs.copy()
alt_ends['chr'] = alt_ends['exon'].str.split('_',expand=True)[0]
alt_ends = alt_ends[['chr','str','gene_name','transcript_id',
                                   'exon_start',
                                   'exon_end',
                                   'exon_end_coding',
                                   'flank_exon_start','flank_exon_end',
                         'stop_rel_position']].drop_duplicates()
alt_ends = alt_ends.sort_values('stop_rel_position').reset_index(drop=True)

In [32]:
Counter(alt_ends['stop_rel_position'])

Counter({'stop_downst': 561, 'stop_upst': 159, 'stop_within': 293})

In [33]:
# we consider protein-coding alternative ends of junctions as a control
sites_in_coding_trs = exons.loc[(exons['transcript_type']=='protein_coding')&(exons['downst_junction']!='')]
temp_coding_trs = exons.loc[(exons['transcript_type']=='protein_coding')&(exons['upst_junction']!='')]
sites_in_coding_trs = pd.merge(sites_in_coding_trs,
                            temp_coding_trs[['gene_name','transcript_id',
                                          'exon_end','upst_junction']].rename(columns={'upst_junction':'downst_junction',
                                                                                    'exon_end':'flank_exon_end'}),
how='inner',on=['gene_name','transcript_id','downst_junction'])

sites_in_coding_trs = sites_in_coding_trs[['exon','gene_name','transcript_id','stop_start',
                  'stop_end','exon_start','exon_end',
                  'downst_junction','flank_exon_end']].drop_duplicates().reset_index(drop=True)
sites_in_coding_trs['flank_exon_start'] = sites_in_coding_trs['downst_junction'].str.split('_',expand=True)[1].astype('int')
sites_in_coding_trs['flank_exon_end']  = sites_in_coding_trs['flank_exon_end'].astype('int')
sites_in_coding_trs = sites_in_coding_trs.drop(['downst_junction'],1).drop_duplicates().reset_index(drop=True)

sites_in_coding_trs = pd.merge(sites_in_coding_trs,
                               sites_in_coding_trs[['gene_name','exon_start',
                     'exon_end','flank_exon_end',
                     'flank_exon_start']].drop_duplicates().rename(columns={'exon_end':'exon_end_alt'}),
how='left',on=['gene_name','exon_start','flank_exon_end','flank_exon_start'])

sites_in_coding_trs = sites_in_coding_trs.loc[sites_in_coding_trs['exon_end']!=sites_in_coding_trs['exon_end_alt']]

temp = alt_ends[['gene_name','exon_start','exon_end','exon_end_coding',
                 'flank_exon_start','flank_exon_end']].drop_duplicates().rename(columns={'exon_end_coding':'exon_end_alt'})
temp['nmd_event']=1

sites_in_coding_trs = pd.merge(sites_in_coding_trs,temp,
how='left',on=['gene_name','exon_start','exon_end','exon_end_alt','flank_exon_start','flank_exon_end'])
sites_in_coding_trs['nmd_event'] = sites_in_coding_trs['nmd_event'].fillna(0).astype('int')
sites_in_coding_trs = sites_in_coding_trs.loc[sites_in_coding_trs['nmd_event']==0]

sites_in_coding_trs['str'] = sites_in_coding_trs['exon'].str.split('_',expand=True)[3]
sites_in_coding_trs['chr'] = sites_in_coding_trs['exon'].str.split('_',expand=True)[0]

def get_stop_rel_position_site(x):
    stop_rel_position = ''
    min_end = min([x['exon_end_alt'],x['exon_end']])
    max_end = max([x['exon_end_alt'],x['exon_end']])
    if ((min_end<=x['stop_start'] and x['stop_start']<=max_end) or (min_end<=x['stop_end'] and x['stop_end']<=max_end)):
        stop_rel_position = 'stop_within'
    elif x['str']=='+':
        if x['stop_start']>=x['flank_exon_start']:
            stop_rel_position = 'stop_downst'
        elif x['stop_end']<=min_end:
            stop_rel_position = 'stop_upst'
    elif x['str']=='-':
        if x['stop_start']>=x['flank_exon_start']:
            stop_rel_position = 'stop_upst'
        elif x['stop_end']<=min_end:
            stop_rel_position = 'stop_downst'
    return stop_rel_position

sites_in_coding_trs['stop_rel_position'] = sites_in_coding_trs.apply(lambda x:get_stop_rel_position_site(x),1)
alt_ends_coding = sites_in_coding_trs[['chr','str','gene_name','exon_start','exon_end','exon_end_alt',
                     'flank_exon_start','flank_exon_end','stop_rel_position']].drop_duplicates()
alt_ends_coding['min_exon_end'] = alt_ends_coding[['exon_end','exon_end_alt']].min(1)
alt_ends_coding['max_exon_end'] = alt_ends_coding[['exon_end','exon_end_alt']].max(1)
alt_ends_coding = alt_ends_coding[['chr','str','exon_start','gene_name','min_exon_end','max_exon_end',
                                       'flank_exon_start','flank_exon_end','stop_rel_position']].drop_duplicates()

In [34]:
len(alt_ends_coding)

2564

In [35]:
alt_ends_coding.to_csv('./misc/hg19_alt_ends_coding.tsv', sep=str('\t'),header=True,index=None)

In [36]:
# We consider alternative starts of junctions regardeless of the strand
# (so, disregarding the classification as 5' or 3' at this step)
sites_in_nmd_trs = exons.loc[(exons['transcript_type']=='nonsense_mediated_decay')&(exons['upst_junction']!='')]

temp_nmd_trs = exons.loc[(exons['transcript_type']=='nonsense_mediated_decay')&(exons['downst_junction']!='')]
sites_in_nmd_trs = pd.merge(sites_in_nmd_trs,
                            temp_nmd_trs[['gene_name',
                                          'transcript_id',
                                          'exon_start',
                                          'downst_junction']].rename(columns={'downst_junction':'upst_junction',
                                                                            'exon_start':'flank_exon_start'}),
how='inner',on=['gene_name','transcript_id','upst_junction'])

sites_in_nmd_trs = sites_in_nmd_trs[['exon','gene_name','transcript_id','stop_start',
                  'stop_end','exon_start','exon_end',
                  'upst_junction','flank_exon_start']].drop_duplicates().reset_index(drop=True)
sites_in_nmd_trs['flank_exon_end'] = sites_in_nmd_trs['upst_junction'].str.split('_',expand=True)[0].astype('int')
sites_in_nmd_trs['flank_exon_start']  = sites_in_nmd_trs['flank_exon_start'].astype('int')

sites_in_nmd_trs = sites_in_nmd_trs.drop(['upst_junction'],1).drop_duplicates().reset_index(drop=True)

sites_in_coding_trs = exons.loc[(exons['transcript_type']=='protein_coding')&(exons['upst_junction']!='')]
temp_coding_trs = exons.loc[(exons['transcript_type']=='protein_coding')&(exons['downst_junction']!='')]
sites_in_coding_trs = pd.merge(sites_in_coding_trs,
                            temp_coding_trs[['gene_name','transcript_id',
                                          'exon_start','downst_junction']].rename(columns={'downst_junction':'upst_junction',
                                                                                    'exon_start':'flank_exon_start'}),
how='inner',on=['gene_name','transcript_id','upst_junction'])

sites_in_coding_trs = sites_in_coding_trs[['exon','gene_name','transcript_id','stop_start',
                  'stop_end','exon_start','exon_end',
                  'upst_junction','flank_exon_start']].drop_duplicates().reset_index(drop=True)
sites_in_coding_trs['flank_exon_end'] = sites_in_coding_trs['upst_junction'].str.split('_',expand=True)[0].astype('int')
sites_in_coding_trs['flank_exon_start']  = sites_in_coding_trs['flank_exon_start'].astype('int')
sites_in_coding_trs = sites_in_coding_trs.drop(['upst_junction'],1).drop_duplicates().reset_index(drop=True)

sites_in_coding_trs['coding_site_exists']=1
sites_in_nmd_trs = pd.merge(sites_in_nmd_trs,
                            sites_in_coding_trs[['gene_name','exon_start','exon_end',
                                      'flank_exon_start','flank_exon_end','coding_site_exists']].drop_duplicates(),
how='left',
on=['gene_name','exon_start','exon_end','flank_exon_start','flank_exon_end'])
sites_in_nmd_trs['coding_site_exists'] = sites_in_nmd_trs['coding_site_exists'].fillna(0).astype('int')

sites_in_nmd_trs = sites_in_nmd_trs.loc[sites_in_nmd_trs['coding_site_exists']==0].drop('coding_site_exists',1)

sites_in_nmd_trs = pd.merge(sites_in_nmd_trs,
                            sites_in_coding_trs[['gene_name','exon_start','exon_end',
                    'flank_exon_start','flank_exon_end']].drop_duplicates().rename(columns={'exon_start':'exon_start_coding'}),
how='left',
on=['gene_name','exon_end','flank_exon_start','flank_exon_end'])

sites_in_nmd_trs = sites_in_nmd_trs.loc[~sites_in_nmd_trs['exon_start_coding'].isna()]
sites_in_nmd_trs['exon_start_coding'] = sites_in_nmd_trs['exon_start_coding'].astype('int')
sites_in_nmd_trs = sites_in_nmd_trs.loc[sites_in_nmd_trs['exon_start_coding']!=sites_in_nmd_trs['exon_start']]


sites_in_nmd_trs_2 = exons.loc[(exons['transcript_type']=='nonsense_mediated_decay')&(exons['upst_junction']!='')]
temp_nmd_trs = exons.loc[(exons['transcript_type']=='nonsense_mediated_decay')&(exons['downst_junction']!='')]
sites_in_nmd_trs_2 = pd.merge(sites_in_nmd_trs_2,
                            temp_nmd_trs[['gene_name',
                                          'transcript_id',
                                          'exon_start',
                                          'downst_junction']].rename(columns={'downst_junction':'upst_junction',
                                                                            'exon_start':'flank_exon_start'}),
how='inner',on=['gene_name','transcript_id','upst_junction'])

sites_in_nmd_trs_2 = sites_in_nmd_trs_2[['gene_name',
                                         'exon_start','exon_end',
                                      'upst_junction','flank_exon_start']].drop_duplicates().reset_index(drop=True)
sites_in_nmd_trs_2['flank_exon_end'] = sites_in_nmd_trs_2['upst_junction'].str.split('_',expand=True)[0].astype('int')
sites_in_nmd_trs_2['flank_exon_start']  = sites_in_nmd_trs_2['flank_exon_start'].astype('int')
sites_in_nmd_trs_2 = sites_in_nmd_trs_2.drop(['upst_junction'],1).drop_duplicates().reset_index(drop=True)
sites_in_nmd_trs_2.rename(columns = {'exon_start':'exon_start_coding'},inplace=True)
sites_in_nmd_trs_2['alt_nmd_site_exists'] = 1

sites_in_nmd_trs = pd.merge(sites_in_nmd_trs,
                            sites_in_nmd_trs_2[['gene_name','flank_exon_end','exon_start_coding','alt_nmd_site_exists']].drop_duplicates(),
                           how='left',on=['gene_name','flank_exon_end','exon_start_coding'])
sites_in_nmd_trs['alt_nmd_site_exists'] = sites_in_nmd_trs['alt_nmd_site_exists'].fillna(0).astype('int')
sites_in_nmd_trs = sites_in_nmd_trs.loc[sites_in_nmd_trs['alt_nmd_site_exists']==0].drop('alt_nmd_site_exists',1)

sites_in_nmd_trs[['stop_start', 'stop_end',
       'exon_start', 'exon_end', 'flank_exon_end', 'flank_exon_start',
       'exon_start_coding']] = sites_in_nmd_trs[['stop_start', 'stop_end',
       'exon_start', 'exon_end', 'flank_exon_end', 'flank_exon_start',
       'exon_start_coding']].astype('int')

sites_in_nmd_trs['str'] = sites_in_nmd_trs['exon'].str.split('_',expand=True)[3]

def get_stop_rel_position_site(x):
    stop_rel_position = ''
    min_start = min([x['exon_start_coding'],x['exon_start']])
    max_start = max([x['exon_start_coding'],x['exon_start']])
    if (x['exon_start_coding']>x['exon_start']) and ((x['exon_start']<=x['stop_start'] and x['stop_start']<=x['exon_start_coding']) or (x['exon_start']<=x['stop_end'] and x['stop_end']<=x['exon_start_coding'])):
        stop_rel_position = 'stop_within'
    elif x['str']=='+':
        if x['stop_start']>=max_start:
            stop_rel_position = 'stop_downst'
        elif x['stop_end']<=x['flank_exon_end']:
            stop_rel_position = 'stop_upst'
    elif x['str']=='-':
        if x['stop_start']>=max_start:
            stop_rel_position = 'stop_upst'
        elif x['stop_end']<=x['flank_exon_end']:
            stop_rel_position = 'stop_downst'
    return stop_rel_position

sites_in_nmd_trs['stop_rel_position'] = sites_in_nmd_trs.apply(lambda x:get_stop_rel_position_site(x),1)
alt_starts = sites_in_nmd_trs.copy()
alt_starts['chr'] = alt_starts['exon'].str.split('_',expand=True)[0]
alt_starts = alt_starts[['chr','str','gene_name','transcript_id','exon_start','exon_start_coding',
                    'exon_end','flank_exon_start','flank_exon_end','stop_rel_position']].drop_duplicates()
alt_starts = alt_starts.sort_values('stop_rel_position').reset_index(drop=True)

In [37]:
Counter(alt_starts['stop_rel_position'])

Counter({'stop_downst': 639, 'stop_upst': 137, 'stop_within': 309})

In [38]:
# we consider protein-coding alternative starts of junctions as a control
sites_in_coding_trs = exons.loc[(exons['transcript_type']=='protein_coding')&(exons['upst_junction']!='')]
temp_coding_trs = exons.loc[(exons['transcript_type']=='protein_coding')&(exons['downst_junction']!='')]
sites_in_coding_trs = pd.merge(sites_in_coding_trs,
                            temp_coding_trs[['gene_name','transcript_id',
                                          'exon_start','downst_junction']].rename(columns={'downst_junction':'upst_junction',
                                                                                    'exon_start':'flank_exon_start'}),
how='inner',on=['gene_name','transcript_id','upst_junction'])

sites_in_coding_trs = sites_in_coding_trs[['exon','gene_name','transcript_id','stop_start',
                  'stop_end','exon_start','exon_end',
                  'upst_junction','flank_exon_start']].drop_duplicates().reset_index(drop=True)
sites_in_coding_trs['flank_exon_end'] = sites_in_coding_trs['upst_junction'].str.split('_',expand=True)[0].astype('int')
sites_in_coding_trs['flank_exon_start']  = sites_in_coding_trs['flank_exon_start'].astype('int')
sites_in_coding_trs = sites_in_coding_trs.drop(['upst_junction'],1).drop_duplicates().reset_index(drop=True)

sites_in_coding_trs = pd.merge(sites_in_coding_trs,
                               sites_in_coding_trs[['gene_name','exon_start',
                     'exon_end','flank_exon_end',
                     'flank_exon_start']].drop_duplicates().rename(columns={'exon_start':'exon_start_alt'}),
how='left',on=['gene_name','exon_end','flank_exon_end','flank_exon_start'])

sites_in_coding_trs = sites_in_coding_trs.loc[sites_in_coding_trs['exon_start']!=sites_in_coding_trs['exon_start_alt']]

temp = alt_starts[['gene_name','exon_start','exon_end','exon_start_coding',
                 'flank_exon_start','flank_exon_end']].drop_duplicates().rename(columns={'exon_start_coding':'exon_start_alt'})
temp['nmd_event']=1

sites_in_coding_trs = pd.merge(sites_in_coding_trs,temp,
how='left',on=['gene_name','exon_start','exon_end','exon_start_alt','flank_exon_start','flank_exon_end'])
sites_in_coding_trs['nmd_event'] = sites_in_coding_trs['nmd_event'].fillna(0).astype('int')
sites_in_coding_trs = sites_in_coding_trs.loc[sites_in_coding_trs['nmd_event']==0]

sites_in_coding_trs['str'] = sites_in_coding_trs['exon'].str.split('_',expand=True)[3]
sites_in_coding_trs['chr'] = sites_in_coding_trs['exon'].str.split('_',expand=True)[0]

def get_stop_rel_position_site(x):
    stop_rel_position = ''
    min_start = min([x['exon_start_alt'],x['exon_start']])
    max_start = max([x['exon_start_alt'],x['exon_start']])
    if ((min_start<=x['stop_start'] and x['stop_start']<=max_start) or (min_start<=x['stop_end'] and x['stop_end']<=max_start)):
        stop_rel_position = 'stop_within'
    elif x['str']=='+':
        if x['stop_start']>=max_start:
            stop_rel_position = 'stop_downst'
        elif x['stop_end']<=x['flank_exon_end']:
            stop_rel_position = 'stop_upst'
    elif x['str']=='-':
        if x['stop_start']>=max_start:
            stop_rel_position = 'stop_upst'
        elif x['stop_end']<=x['flank_exon_end']:
            stop_rel_position = 'stop_downst'
    return stop_rel_position

sites_in_coding_trs['stop_rel_position'] = sites_in_coding_trs.apply(lambda x:get_stop_rel_position_site(x),1)
alt_starts_coding = sites_in_coding_trs[['chr','str','gene_name','exon_start','exon_end','exon_start_alt',
                     'flank_exon_start','flank_exon_end','stop_rel_position']].drop_duplicates()
alt_starts_coding['min_exon_start'] = alt_starts_coding[['exon_start','exon_start_alt']].min(1)
alt_starts_coding['max_exon_start'] = alt_starts_coding[['exon_start','exon_start_alt']].max(1)
alt_starts_coding = alt_starts_coding[['chr','str','gene_name','min_exon_start','max_exon_start','exon_end','flank_exon_start','flank_exon_end','stop_rel_position']].drop_duplicates()

In [39]:
len(alt_starts_coding)

2584

In [40]:
alt_starts_coding.to_csv('./misc/hg19_alt_starts_coding.tsv', sep=str('\t'),header=True,index=None)

## Intron retention (IR)

In [41]:
# we identify introns that are specifically retained in NMD transcripts
exons_in_nmd_trs = exons.loc[(exons['transcript_type']=='nonsense_mediated_decay')]

exons_in_coding_trs = exons.loc[(exons['transcript_type']=='protein_coding')&(exons['downst_junction']!='')]
temp = exons.loc[(exons['transcript_type']=='protein_coding')&(exons['upst_junction']!='')]
exons_in_coding_trs = pd.merge(exons_in_coding_trs,
temp[['gene_name',
      'transcript_id',
      'exon_end',
      'upst_junction']].drop_duplicates().rename(columns={'upst_junction':'downst_junction',
                                                          'exon_end':'downst_exon_end'}),
how='left',on=['gene_name','transcript_id','downst_junction'])
exons_in_coding_trs['exon_end_coding'] = exons_in_coding_trs['exon_end']
exons_in_coding_trs['downst_exon_start_coding'] = exons_in_coding_trs['downst_junction'].str.split('_',expand=True)[1].astype('int')

exons_in_coding_trs = exons_in_coding_trs[['gene_name','exon_start',
                     'exon_end_coding','downst_exon_start_coding',
                     'downst_exon_end']].drop_duplicates().reset_index(drop=True).rename(columns={'downst_exon_end':'exon_end'})

temp = exons.loc[(exons['transcript_type']=='protein_coding')][['gene_name','exon_start','exon_end']].drop_duplicates()
temp['IR_exists_in_coding']=1
exons_in_coding_trs = pd.merge(exons_in_coding_trs,temp,how='left',on=['gene_name','exon_start','exon_end'])
exons_in_coding_trs['IR_exists_in_coding'] = exons_in_coding_trs['IR_exists_in_coding'].fillna(0).astype('int')
exons_in_coding_trs = exons_in_coding_trs.loc[exons_in_coding_trs['IR_exists_in_coding']==0].drop('IR_exists_in_coding',1)

exons_in_nmd_trs = pd.merge(exons_in_nmd_trs,exons_in_coding_trs,how='left',on=['gene_name','exon_start','exon_end'])
exons_in_nmd_trs = exons_in_nmd_trs.loc[(~exons_in_nmd_trs['exon_end_coding'].isna())&(~exons_in_nmd_trs['downst_exon_start_coding'].isna())]
exons_in_nmd_trs[['exon_end_coding','downst_exon_start_coding']] = exons_in_nmd_trs[['exon_end_coding','downst_exon_start_coding']].astype('int')
exons_in_nmd_trs['chr'] = exons_in_nmd_trs['exon'].str.split('_',expand=True)[0]
exons_in_nmd_trs['str'] = exons_in_nmd_trs['exon'].str.split('_',expand=True)[3]


exons_in_nmd_trs_2 = exons.loc[(exons['transcript_type']=='nonsense_mediated_decay')&(exons['downst_junction']!='')]
temp = exons.loc[(exons['transcript_type']=='nonsense_mediated_decay')&(exons['upst_junction']!='')]
exons_in_nmd_trs_2 = pd.merge(exons_in_nmd_trs_2,
temp[['gene_name',
      'transcript_id',
      'exon_end',
      'upst_junction']].drop_duplicates().rename(columns={'upst_junction':'downst_junction',
                                                          'exon_end':'downst_exon_end'}),
how='left',on=['gene_name','transcript_id','downst_junction'])
exons_in_nmd_trs_2['exon_end_coding'] = exons_in_nmd_trs_2['exon_end']
exons_in_nmd_trs_2['downst_exon_start_coding'] = exons_in_nmd_trs_2['downst_junction'].str.split('_',expand=True)[1].astype('int')

exons_in_nmd_trs_2 = exons_in_nmd_trs_2[['gene_name','exon_start',
                     'exon_end_coding','downst_exon_start_coding',
                     'downst_exon_end']].drop_duplicates().reset_index(drop=True).rename(columns={'downst_exon_end':'exon_end'})
exons_in_nmd_trs_2['ID_nmd_exists'] = 1

exons_in_nmd_trs = pd.merge(exons_in_nmd_trs,exons_in_nmd_trs_2,how='left',on=['gene_name','exon_start','exon_end_coding','downst_exon_start_coding','exon_end'])
exons_in_nmd_trs['ID_nmd_exists'] = exons_in_nmd_trs['ID_nmd_exists'].fillna(0).astype('int')
exons_in_nmd_trs = exons_in_nmd_trs.loc[exons_in_nmd_trs['ID_nmd_exists']==0].drop('ID_nmd_exists',1)

def get_stop_rel_position_intron(x):
    stop_rel_position = ''
    if ((x['exon_end_coding']<x['stop_start'] and x['stop_start']<x['downst_exon_start_coding']) or (x['exon_end_coding']<x['stop_end'] and x['stop_end']<x['downst_exon_start_coding'])):
        stop_rel_position = 'stop_within'
    elif x['str']=='+':
        if x['stop_start']>=x['downst_exon_start_coding']:
            stop_rel_position = 'stop_downst'
        elif x['stop_end']<=x['exon_end_coding']:
            stop_rel_position = 'stop_upst'
    elif x['str']=='-':
        if x['stop_start']>=x['downst_exon_start_coding']:
            stop_rel_position = 'stop_upst'
        elif x['stop_end']<=x['exon_end_coding']:
            stop_rel_position = 'stop_downst'
    return stop_rel_position

exons_in_nmd_trs['stop_rel_position'] = exons_in_nmd_trs.apply(lambda x:get_stop_rel_position_intron(x),1)

intron_retention = exons_in_nmd_trs[['chr','str','gene_name','transcript_id','exon_start',
                  'exon_end_coding','downst_exon_start_coding',
                  'exon_end','stop_rel_position']].drop_duplicates().reset_index(drop=True)

In [42]:
Counter(intron_retention['stop_rel_position'])

Counter({'stop_upst': 127, 'stop_downst': 28, 'stop_within': 68})

In [44]:
# we identify introns that are specifically detained (spliced) in NMD transcripts
exons_in_nmd_trs = exons.loc[(exons['transcript_type']=='nonsense_mediated_decay')&(exons['downst_junction']!='')]
temp = exons.loc[(exons['transcript_type']=='nonsense_mediated_decay')&(exons['upst_junction']!='')]

exons_in_nmd_trs = pd.merge(exons_in_nmd_trs,
    temp[['gene_name',
          'transcript_id',
          'exon_end',
          'upst_junction']].drop_duplicates().rename(columns={'upst_junction':'downst_junction',
                                                              'exon_end':'downst_exon_end'}),
    how='left',on=['gene_name','transcript_id','downst_junction'])
exons_in_nmd_trs['upst_exon_end'] = exons_in_nmd_trs['exon_end']
exons_in_nmd_trs['upst_exon_start'] = exons_in_nmd_trs['exon_start']
exons_in_nmd_trs['downst_exon_start'] = exons_in_nmd_trs['downst_junction'].str.split('_',expand=True)[1].astype('int')

exons_in_nmd_trs['chr'] = exons_in_nmd_trs['exon'].str.split('_',expand=True)[0]
exons_in_nmd_trs['str'] = exons_in_nmd_trs['exon'].str.split('_',expand=True)[3]

exons_in_nmd_trs = exons_in_nmd_trs[['chr','str','gene_name','transcript_id',
                                     'upst_exon_start','upst_exon_end',
                                     'downst_exon_start','downst_exon_end',
                     'stop_start','stop_end']].drop_duplicates().reset_index(drop=True)

exons_in_coding_trs = exons.loc[(exons['transcript_type']=='protein_coding')&(exons['downst_junction']!='')]
temp = exons.loc[(exons['transcript_type']=='protein_coding')&(exons['upst_junction']!='')]
exons_in_coding_trs = pd.merge(exons_in_coding_trs,
temp[['gene_name',
      'transcript_id',
      'exon_end',
      'upst_junction']].drop_duplicates().rename(columns={'upst_junction':'downst_junction',
                                                          'exon_end':'downst_exon_end'}),
how='left',on=['gene_name','transcript_id','downst_junction'])
exons_in_coding_trs['upst_exon_end'] = exons_in_coding_trs['exon_end']
exons_in_coding_trs['downst_exon_start'] = exons_in_coding_trs['downst_junction'].str.split('_',expand=True)[1].astype('int')
exons_in_coding_trs['upst_exon_start'] = exons_in_coding_trs['exon_start']

exons_in_coding_trs = exons_in_coding_trs[['gene_name',
                                     'upst_exon_start','upst_exon_end',
                                     'downst_exon_start','downst_exon_end']].drop_duplicates().reset_index(drop=True)
exons_in_coding_trs['ID_exists_in_coding'] = 1
exons_in_nmd_trs = pd.merge(exons_in_nmd_trs,exons_in_coding_trs,how='left',on=['gene_name','upst_exon_start','upst_exon_end',
                                                    'downst_exon_start','downst_exon_end'])
exons_in_nmd_trs['ID_exists_in_coding'] = exons_in_nmd_trs['ID_exists_in_coding'].fillna(0).astype('int')
exons_in_nmd_trs = exons_in_nmd_trs.loc[exons_in_nmd_trs['ID_exists_in_coding']==0]

temp = exons.loc[(exons['transcript_type']=='protein_coding')]
temp = temp[['gene_name',
             'exon_start',
             'exon_end']].drop_duplicates().reset_index(drop=True).rename(columns={'exon_start':'upst_exon_start',
                                                                                   'exon_end':'downst_exon_end'})
temp['IR_exists_in_coding']=1
exons_in_nmd_trs = pd.merge(exons_in_nmd_trs,temp,how='left',on=['gene_name','upst_exon_start','downst_exon_end'])
exons_in_nmd_trs['IR_exists_in_coding'] = exons_in_nmd_trs['IR_exists_in_coding'].fillna(0).astype('int')
exons_in_nmd_trs = exons_in_nmd_trs.loc[exons_in_nmd_trs['IR_exists_in_coding']==1]

temp = exons.loc[(exons['transcript_type']=='nonsense_mediated_decay')]
temp = temp[['gene_name',
             'exon_start',
             'exon_end']].drop_duplicates().reset_index(drop=True).rename(columns={'exon_start':'upst_exon_start',
                                                                                   'exon_end':'downst_exon_end'})
temp['IR_exists_in_nmd']=1
exons_in_nmd_trs = pd.merge(exons_in_nmd_trs,temp,how='left',on=['gene_name','upst_exon_start','downst_exon_end'])
exons_in_nmd_trs['IR_exists_in_nmd'] = exons_in_nmd_trs['IR_exists_in_nmd'].fillna(0).astype('int')
exons_in_nmd_trs = exons_in_nmd_trs.loc[exons_in_nmd_trs['IR_exists_in_nmd']==0]

exons_in_nmd_trs['stop_len'] = exons_in_nmd_trs['stop_end']-exons_in_nmd_trs['stop_start']+1

def get_stop_rel_position_intron(x):
    stop_rel_position = ''
    if (x['stop_len']<3) and (x['stop_start']<=x['upst_exon_end'] and (x['stop_start']+2)>=x['upst_exon_end']):
        stop_rel_position = 'stop_within'
    elif x['str']=='+':
        if x['stop_start']>=x['downst_exon_start']:
            stop_rel_position = 'stop_downst'
        elif x['stop_end']<=x['upst_exon_end']:
            stop_rel_position = 'stop_upst'          
    elif x['str']=='-':
        if x['stop_start']>=x['downst_exon_start']:
            stop_rel_position = 'stop_upst'
        elif x['stop_end']<=x['upst_exon_end']:
            stop_rel_position = 'stop_downst' 
    return stop_rel_position

exons_in_nmd_trs['stop_rel_position'] = exons_in_nmd_trs.apply(lambda x:get_stop_rel_position_intron(x),1)

intron_detention = exons_in_nmd_trs[['chr','str','gene_name',
                                     'transcript_id','upst_exon_start',
                  'upst_exon_end','downst_exon_start',
                'downst_exon_end','stop_rel_position']].drop_duplicates().reset_index(drop=True)

In [45]:
Counter(intron_detention['stop_rel_position'])

Counter({'stop_upst': 101, 'stop_downst': 70, 'stop_within': 1})

In [47]:
# contol set of protein-coding IR events
exons_in_coding_trs = exons.loc[(exons['transcript_type']=='protein_coding')&(exons['downst_junction']!='')]
temp = exons.loc[(exons['transcript_type']=='protein_coding')&(exons['upst_junction']!='')]
exons_in_coding_trs = pd.merge(exons_in_coding_trs,
temp[['gene_name',
      'transcript_id',
      'exon_end',
      'upst_junction']].drop_duplicates().rename(columns={'upst_junction':'downst_junction',
                                                          'exon_end':'downst_exon_end'}),
how='left',on=['gene_name','transcript_id','downst_junction'])
exons_in_coding_trs['upst_exon_end'] = exons_in_coding_trs['exon_end']
exons_in_coding_trs['downst_exon_start'] = exons_in_coding_trs['downst_junction'].str.split('_',expand=True)[1].astype('int')
exons_in_coding_trs['upst_exon_start'] = exons_in_coding_trs['exon_start']

exons_in_coding_trs['chr'] = exons_in_coding_trs['exon'].str.split('_',expand=True)[0]
exons_in_coding_trs['str'] = exons_in_coding_trs['exon'].str.split('_',expand=True)[3]

temp = exons.loc[(exons['transcript_type']=='protein_coding')][['gene_name','exon_start','exon_end']].drop_duplicates()
temp.rename(columns={'exon_start':'upst_exon_start','exon_end':'downst_exon_end'},inplace=True)
temp['IR_exists_in_coding']=1
exons_in_coding_trs = pd.merge(exons_in_coding_trs,temp,how='left',on=['gene_name','upst_exon_start','downst_exon_end'])
exons_in_coding_trs['IR_exists_in_coding'] = exons_in_coding_trs['IR_exists_in_coding'].fillna(0).astype('int')
exons_in_coding_trs = exons_in_coding_trs.loc[exons_in_coding_trs['IR_exists_in_coding']==1].drop('IR_exists_in_coding',1)

temp = exons.loc[(exons['transcript_type']=='nonsense_mediated_decay')][['gene_name','exon_start','exon_end']].drop_duplicates()
temp.rename(columns={'exon_start':'upst_exon_start','exon_end':'downst_exon_end'},inplace=True)
temp['IR_exists_in_nmd']=1
exons_in_coding_trs = pd.merge(exons_in_coding_trs,temp,how='left',on=['gene_name','upst_exon_start','downst_exon_end'])
exons_in_coding_trs['IR_exists_in_nmd'] = exons_in_coding_trs['IR_exists_in_nmd'].fillna(0).astype('int')
exons_in_coding_trs = exons_in_coding_trs.loc[exons_in_coding_trs['IR_exists_in_nmd']==0].drop('IR_exists_in_nmd',1)


exons_in_nmd_trs = exons.loc[(exons['transcript_type']=='nonsense_mediated_decay')&(exons['downst_junction']!='')]
temp = exons.loc[(exons['transcript_type']=='nonsense_mediated_decay')&(exons['upst_junction']!='')]
exons_in_nmd_trs = pd.merge(exons_in_nmd_trs,
temp[['gene_name',
      'transcript_id',
      'exon_end',
      'upst_junction']].drop_duplicates().rename(columns={'upst_junction':'downst_junction',
                                                          'exon_end':'downst_exon_end'}),
how='left',on=['gene_name','transcript_id','downst_junction'])
exons_in_nmd_trs['upst_exon_end'] = exons_in_nmd_trs['exon_end']
exons_in_nmd_trs['downst_exon_start'] = exons_in_nmd_trs['downst_junction'].str.split('_',expand=True)[1].astype('int')
exons_in_nmd_trs['upst_exon_start'] = exons_in_nmd_trs['exon_start']
exons_in_nmd_trs['ID_exists_in_nmd'] = 1

exons_in_coding_trs = pd.merge(exons_in_coding_trs,
                               exons_in_nmd_trs[['gene_name','upst_exon_end',
                                                 'downst_exon_start','ID_exists_in_nmd']].drop_duplicates(),
                               how='left',on=['gene_name','upst_exon_end','downst_exon_start'])
exons_in_coding_trs['ID_exists_in_nmd'] = exons_in_coding_trs['ID_exists_in_nmd'].fillna(0).astype('int')
exons_in_coding_trs = exons_in_coding_trs.loc[exons_in_coding_trs['ID_exists_in_nmd']==0]

exons_in_coding_trs = exons_in_coding_trs[['chr','str','gene_name','transcript_id','stop_start','stop_end',
                                           'upst_exon_start','upst_exon_end',
                     'downst_exon_start','downst_exon_end']].drop_duplicates().reset_index(drop=True)

def get_stop_rel_position_intron(x):
    stop_rel_position = ''
    if ((x['upst_exon_end']<x['stop_start'] and x['stop_start']<x['downst_exon_start']) or (x['upst_exon_end']<x['stop_end'] and x['stop_end']<x['downst_exon_start'])):
        stop_rel_position = 'stop_within'
    elif x['str']=='+':
        if x['stop_start']>=x['downst_exon_start']:
            stop_rel_position = 'stop_downst'
        elif x['stop_end']<=x['upst_exon_end']:
            stop_rel_position = 'stop_upst'
    elif x['str']=='-':
        if x['stop_start']>=x['downst_exon_start']:
            stop_rel_position = 'stop_upst'
        elif x['stop_end']<=x['upst_exon_end']:
            stop_rel_position = 'stop_downst'
    return stop_rel_position

exons_in_coding_trs['stop_rel_position'] = exons_in_coding_trs.apply(lambda x:get_stop_rel_position_intron(x),1)
intron_retention_coding = exons_in_coding_trs[['chr','str','gene_name','upst_exon_start',
                                               'upst_exon_end','downst_exon_start','downst_exon_end','stop_rel_position']].drop_duplicates().reset_index(drop=True)

In [48]:
len(intron_retention_coding)

758

In [49]:
intron_retention_coding.to_csv('./misc/hg19_intron_retention_coding.tsv', sep=str('\t'),header=True,index=None)

## We group identified NMD-inducing events over transcripts  

In [50]:
# We select for each transcript the most likely events that cause NMD, 
# from the point of their position relative to the stop codon

poison_exons['event']='PE'
essential_exons['event']='EE'
alt_ends['event'] = 'alt_end'
alt_starts['event'] = 'alt_start'
intron_retention['event'] = 'IR'
intron_detention['event'] = 'ID'

summary = pd.concat([poison_exons[['transcript_id','stop_rel_position','event']].drop_duplicates(),
essential_exons[['transcript_id','stop_rel_position','event']].drop_duplicates(),
alt_ends[['transcript_id','stop_rel_position','event']].drop_duplicates(),
alt_starts[['transcript_id','stop_rel_position','event']].drop_duplicates(),
intron_retention[['transcript_id','stop_rel_position','event']].drop_duplicates(),
intron_detention[['transcript_id','stop_rel_position','event']].drop_duplicates()
                    ])

summary['t']=1
gr = summary.groupby(['transcript_id','stop_rel_position']).agg({'t':sum}).reset_index()
def get_rank(x):
    rank=4
    if x['stop_rel_position']=='stop_within':
        rank=1
    elif x['stop_rel_position']=='stop_downst':
        rank=2
    elif x['stop_rel_position']=='stop_upst':
        rank=3
    return rank
gr['rank'] = gr.apply(lambda x:get_rank(x),1)
gr = gr.sort_values(['transcript_id','rank'],ascending=[True,True]).reset_index(drop=True)
gr = gr.drop_duplicates(['transcript_id'],keep='first')

summary = pd.merge(summary,gr[['transcript_id','stop_rel_position']],how='inner',on=['transcript_id','stop_rel_position'])

poison_exons = pd.merge(poison_exons,summary.loc[summary['event']=='PE'][['transcript_id','stop_rel_position']].drop_duplicates(),
how='inner',on=['transcript_id','stop_rel_position'])

essential_exons = pd.merge(essential_exons,summary.loc[summary['event']=='EE'][['transcript_id','stop_rel_position']].drop_duplicates(),
how='inner',on=['transcript_id','stop_rel_position'])

alt_ends = pd.merge(alt_ends,summary.loc[summary['event']=='alt_end'][['transcript_id','stop_rel_position']].drop_duplicates(),
how='inner',on=['transcript_id','stop_rel_position'])

alt_starts = pd.merge(alt_starts,summary.loc[summary['event']=='alt_start'][['transcript_id','stop_rel_position']].drop_duplicates(),
how='inner',on=['transcript_id','stop_rel_position'])

intron_retention = pd.merge(intron_retention,summary.loc[summary['event']=='IR'][['transcript_id','stop_rel_position']].drop_duplicates(),
how='inner',on=['transcript_id','stop_rel_position'])

intron_detention = pd.merge(intron_detention,summary.loc[summary['event']=='ID'][['transcript_id','stop_rel_position']].drop_duplicates(),
how='inner',on=['transcript_id','stop_rel_position'])

# Now we retain only those transcripts in which there is only one event that potentially causes NMD

summary = pd.concat([poison_exons[['transcript_id','event']],
essential_exons.drop_duplicates(['gene_name','transcript_id','skip_junction'])[['transcript_id','event']],
alt_ends.drop_duplicates(['gene_name','transcript_id',
                          'exon_start','exon_end',
                          'flank_exon_start','flank_exon_end'])[['transcript_id','event']],
alt_starts.drop_duplicates(['gene_name','transcript_id',
                          'exon_start','exon_end',
                          'flank_exon_start','flank_exon_end'])[['transcript_id','event']],
intron_retention.drop_duplicates(['gene_name','transcript_id','exon_start','exon_end'])[['transcript_id','event']],
intron_detention[['transcript_id','event']]])
summary['t']=1

gr = summary.groupby(['transcript_id']).agg({'t':sum}).reset_index()
summary = summary.loc[summary['transcript_id'].isin(list(gr.loc[gr['t']==1]['transcript_id'].unique()))]

In [51]:
Counter(gr['t'])

Counter({1: 6029, 2: 155, 3: 12, 4: 1})

In [52]:
poison_exons = pd.merge(poison_exons,summary.loc[summary['event']=='PE'][['transcript_id']].drop_duplicates(),
how='inner',on=['transcript_id'])

essential_exons = pd.merge(essential_exons,summary.loc[summary['event']=='EE'][['transcript_id']].drop_duplicates(),
how='inner',on=['transcript_id'])

alt_ends = pd.merge(alt_ends,summary.loc[summary['event']=='alt_end'][['transcript_id']].drop_duplicates(),
how='inner',on=['transcript_id'])

alt_starts = pd.merge(alt_starts,summary.loc[summary['event']=='alt_start'][['transcript_id']].drop_duplicates(),
how='inner',on=['transcript_id'])

intron_retention = pd.merge(intron_retention,summary.loc[summary['event']=='IR'][['transcript_id']].drop_duplicates(),
how='inner',on=['transcript_id'])

intron_detention = pd.merge(intron_detention,summary.loc[summary['event']=='ID'][['transcript_id']].drop_duplicates(),
how='inner',on=['transcript_id'])

In [41]:
poison_exons = poison_exons[['chr','str','gene_name','upst_junction','downst_junction','skip_junction','stop_rel_position']].drop_duplicates()
essential_exons = essential_exons[['chr','str','gene_name','upst_junction','downst_junction','skip_junction','stop_rel_position']].drop_duplicates()
alt_ends = alt_ends[['chr','str','gene_name','exon_start','exon_end','exon_end_coding','flank_exon_start','flank_exon_end','stop_rel_position']].drop_duplicates()
alt_starts = alt_starts[['chr','str','gene_name','exon_start','exon_start_coding','exon_end','flank_exon_start','flank_exon_end','stop_rel_position']].drop_duplicates()
intron_retention = intron_retention[['chr','str','gene_name','exon_start','exon_end_coding','downst_exon_start_coding','exon_end','stop_rel_position']].drop_duplicates()
intron_detention = intron_detention[['chr','str','gene_name','upst_exon_start','upst_exon_end','downst_exon_start','downst_exon_end','stop_rel_position']].drop_duplicates()

In [53]:
essential_exons = essential_exons.loc[essential_exons['stop_rel_position']!='stop_upst']
alt_ends = alt_ends.loc[alt_ends['stop_rel_position']!='stop_upst']
alt_starts = alt_starts.loc[alt_starts['stop_rel_position']!='stop_upst']
intron_retention = intron_retention.loc[intron_retention['stop_rel_position']!='stop_upst']

In [54]:
os.system('mkdir -p ./novel_events_catalog/')
poison_exons.to_csv('./novel_events_catalog/hg19_poison_exons.tsv', sep=str('\t'),header=True,index=None)
essential_exons.to_csv('./novel_events_catalog/hg19_essential_exons.tsv', sep=str('\t'),header=True,index=None)
alt_ends.to_csv('./novel_events_catalog/hg19_alt_ends.tsv', sep=str('\t'),header=True,index=None)
alt_starts.to_csv('./novel_events_catalog/hg19_alt_starts.tsv', sep=str('\t'),header=True,index=None)
intron_retention.to_csv('./novel_events_catalog/hg19_intron_retention.tsv', sep=str('\t'),header=True,index=None)
intron_detention.to_csv('./novel_events_catalog/hg19_intron_detention.tsv', sep=str('\t'),header=True,index=None)

In [55]:
len(poison_exons)+len(essential_exons)+len(alt_ends)+len(alt_starts)+len(intron_retention)+len(intron_detention)

5871

In [ ]:
# we check the abundance of alternative 5' (donor) and 3' (acceptor) splice sites 

In [70]:
alt_ends['DA_type'] = (alt_ends['str']=='+').astype('int').astype('str').str.replace('1','D').replace('0','A')
alt_starts['DA_type'] = (alt_starts['str']=='+').astype('int').astype('str').str.replace('1','A').replace('0','D')

In [72]:
alt_ends['t1']=1
temp1 = alt_ends.groupby(['DA_type','stop_rel_position']).agg({'t1':sum}).reset_index()
alt_starts['t2']=1
temp2 = alt_starts.groupby(['DA_type','stop_rel_position']).agg({'t2':sum}).reset_index()
temp1 = pd.merge(temp1,temp2,how='inner',on=['DA_type','stop_rel_position'])
temp1['t']=temp1['t1']+temp1['t2']
temp1

,DA_type,stop_rel_position,t1,t2,t
0,A,stop_downst,547,572,1119
1,A,stop_within,284,310,594
2,D,stop_downst,319,369,688
3,D,stop_within,232,213,445


In [73]:
temp1.groupby('DA_type').agg({'t':sum}).reset_index()

,DA_type,t
0,A,1713
1,D,1133


In [82]:
Counter(intron_detention['stop_rel_position'])

Counter({'stop_upst': 68, 'stop_downst': 77, 'stop_within': 3})

# We quantify $\psi_H-\psi_L$, $\Delta e_g$ and $\Delta e_l$ using GTEx

## validated USEs

In [234]:
leus = pd.read_csv('./misc/Literature_examples_of_unproductive_splicing.txt',delimiter="\t",
                                   index_col=None,header=0)
leus = leus.fillna('')

cols = ['Target','AS event position','AS type','PTC position','rMATS AS type','NMD site upstream hg19','NMD site downstream hg19',
       'NMD junction upstream hg19', 'NMD junction downstream hg19',
       'coding site upstream hg19', 'coding site downstream hg19',
       'coding junction upstream hg19', 'coding junction downstream hg19']
leus_df = leus[cols].drop_duplicates().reset_index(drop=True)
leus_df = leus_df.loc[leus_df['Target']!='']

In [235]:
len(leus_df),len(leus_df['Target'].unique())

(58, 51)

In [278]:
cols = pd.Series(leus_df.columns)
NMD_cols = list(cols.loc[cols.str.contains('NMD ')])
can_cols = list(cols.loc[cols.str.contains('coding ')])
          
GTEX_summary_pooled = aaf.get_GTEX_summary_pooled(leus_df,NMD_cols,can_cols)

done: 0 out of 58
no expression for: CCN1 intron 3
done: 20 out of 58
done: 40 out of 58


In [280]:
GTEX_summary_pooled = pd.DataFrame(GTEX_summary_pooled,columns = ['gene_name','AS event position','psi_L','psi_H',
                                            'log2FC_global','log2FC_local','zscore_global','zscore_local'])
GTEX_summary_pooled['psi_H_L'] = GTEX_summary_pooled['psi_H']-GTEX_summary_pooled['psi_L']
GTEX_summary_pooled = pd.merge(GTEX_summary_pooled,leus_df[['Target','AS event position','AS type']].rename(columns={'Target':'gene_name'}),
how='left',on=['gene_name','AS event position'])

In [281]:
# we need this step because the test was performed in an opposite direction
GTEX_summary_pooled['zscore_global'] = GTEX_summary_pooled['zscore_global']*(-1)
GTEX_summary_pooled['zscore_local'] = GTEX_summary_pooled['zscore_local']*(-1)

## Protein-coding AS events

In [301]:
# protein-coding skipped exons
coding_ce = pd.read_csv('/home/magmir/TASS/NMD_regulation/novel_events_catalog/hg19_cassette_exons_coding.tsv',delimiter="\t",
                                   index_col=None,header=0)
coding_sample = coding_ce.copy()
coding_sample['AS event position'] = coding_sample['chr']+'_'+coding_sample['upst_junction']+'_'+coding_sample['downst_junction']+'_'+coding_sample['str']
coding_sample['skip_junction'] = coding_sample['chr']+'_'+coding_sample['skip_junction']+'_'+coding_sample['str']
coding_sample['upst_junction'] = coding_sample['chr']+'_'+coding_sample['upst_junction']+'_'+coding_sample['str']
coding_sample['downst_junction'] = coding_sample['chr']+'_'+coding_sample['downst_junction']+'_'+coding_sample['str']
coding_sample['NMD_skip_junction'] = coding_sample['skip_junction']
coding_sample.rename(columns={'gene_name':'Target'},inplace=True)
coding_sample = coding_sample.drop_duplicates(['Target','AS event position'])

In [318]:
NMD_cols = ['NMD_skip_junction']
can_cols = ['upst_junction','downst_junction']

a = []
s = 0
while s<len(coding_sample):
    e=s+1000
    temp_df = coding_sample.iloc[s:min(e,len(coding_sample))]
    temp_summary = aaf.get_GTEX_summary_pooled(temp_df,NMD_cols,can_cols)
    a = a+temp_summary
    print(str(s)+':'+str(e)+' done')
    s=s+1000

done: 0 out of 1000
LQ or UQ are zero length: AAAS chr12_53708924_53709119_53709210_53709511_-
LQ or UQ are zero length: AASDH chr4_57244630_57248643_57248763_57250236_-
LQ or UQ are zero length: AASDH chr4_57248763_57250236_57250507_57253528_-
LQ or UQ are zero length: ABCB7 chrX_74296489_74318777_74318896_74332721_-
done: 20 out of 1000
done: 40 out of 1000
done: 60 out of 1000
done: 80 out of 1000
done: 100 out of 1000
done: 140 out of 1000
no expression for: CCN4 chr8_134225386_134232824_134233084_134237633_+
no expression for: CCN4 chr8_134225386_134237633_134237826_134239654_+
done: 160 out of 1000
no expression for: CEP20 chr16_15961373_15967348_15967484_15973661_-
no expression for: CEP20 chr16_15973745_15977865_15978062_15982415_-
done: 180 out of 1000
done: 200 out of 1000
done: 220 out of 1000
done: 240 out of 1000
no expression for: DGLUCY chr14_91666249_91671050_91671169_91681749_+
done: 260 out of 1000
no expression for: DMAC2 chr19_41938313_41939177_41939339_41939442_-
n

In [319]:
a_df = pd.DataFrame(a,columns = ['gene_name','AS event position','psi_L','psi_H',
                                            'log2FC_global','log2FC_local','zscore_global','zscore_local'])

In [320]:
a_df.to_csv('./misc/hg19_coding_CE_skip_summary_2.tsv', sep=str('\t'),header=True,index=None)

In [324]:
# кодирующие альтернативные сайты
hg19_alt_starts_coding = pd.read_csv('./misc/hg19_alt_starts_coding.tsv',delimiter="\t",
                                   index_col=None,header=0)
hg19_alt_ends_coding = pd.read_csv('./misc/hg19_alt_ends_coding.tsv',delimiter="\t",
                                   index_col=None,header=0)

hg19_alt_starts_coding.rename(columns={'gene_name':'Target'},inplace=True)
hg19_alt_ends_coding.rename(columns={'gene_name':'Target'},inplace=True)

hg19_alt_starts_coding['AS event position'] = hg19_alt_starts_coding['chr']+'_'+hg19_alt_starts_coding['flank_exon_start'].astype('str')+\
'_'+hg19_alt_starts_coding['flank_exon_end'].astype('str')+'_'+hg19_alt_starts_coding['min_exon_start'].astype('str')+\
'_'+hg19_alt_starts_coding['max_exon_start'].astype('str')+'_'+hg19_alt_starts_coding['exon_end'].astype('str')+'_'+hg19_alt_starts_coding['str']

hg19_alt_ends_coding['AS event position'] = hg19_alt_ends_coding['chr']+'_'+hg19_alt_ends_coding['exon_start'].astype('str')+\
'_'+hg19_alt_ends_coding['min_exon_end'].astype('str')+'_'+hg19_alt_ends_coding['max_exon_end'].astype('str')+\
'_'+hg19_alt_ends_coding['flank_exon_start'].astype('str')+'_'+hg19_alt_ends_coding['flank_exon_end'].astype('str')+'_'+hg19_alt_ends_coding['str']

hg19_alt_starts_coding = hg19_alt_starts_coding.drop_duplicates(['Target','AS event position'])
hg19_alt_ends_coding = hg19_alt_ends_coding.drop_duplicates(['Target','AS event position'])

hg19_alt_starts_coding['DA_type'] = (hg19_alt_starts_coding['str']=='+').astype('int').astype('str').str.replace('1','A').replace('0','D')
hg19_alt_ends_coding['DA_type'] = (hg19_alt_ends_coding['str']=='+').astype('int').astype('str').str.replace('1','D').replace('0','A')

acc1 = hg19_alt_starts_coding.loc[hg19_alt_starts_coding['DA_type']=='A'].reset_index(drop=True)
acc2 = hg19_alt_ends_coding.loc[hg19_alt_ends_coding['DA_type']=='A'].reset_index(drop=True)

don1 = hg19_alt_starts_coding.loc[hg19_alt_starts_coding['DA_type']=='D'].reset_index(drop=True)
don2 = hg19_alt_ends_coding.loc[hg19_alt_ends_coding['DA_type']=='D'].reset_index(drop=True)

acc1['NMD_junction'] = acc1['chr']+'_'+acc1['flank_exon_end'].astype('str')+'_'+acc1['min_exon_start'].astype('str')+'_'+acc1['str']
acc1['coding_junction'] = acc1['chr']+'_'+acc1['flank_exon_end'].astype('str')+'_'+acc1['max_exon_start'].astype('str')+'_'+acc1['str']

acc2['NMD_junction'] = acc2['chr']+'_'+acc2['max_exon_end'].astype('str')+'_'+acc2['flank_exon_start'].astype('str')+'_'+acc2['str']
acc2['coding_junction'] = acc2['chr']+'_'+acc2['min_exon_end'].astype('str')+'_'+acc2['flank_exon_start'].astype('str')+'_'+acc2['str']

don1['NMD_junction'] = don1['chr']+'_'+don1['flank_exon_end'].astype('str')+'_'+don1['min_exon_start'].astype('str')+'_'+don1['str']
don1['coding_junction'] = don1['chr']+'_'+don1['flank_exon_end'].astype('str')+'_'+don1['max_exon_start'].astype('str')+'_'+don1['str']

don2['NMD_junction'] = don2['chr']+'_'+don2['max_exon_end'].astype('str')+'_'+don2['flank_exon_start'].astype('str')+'_'+don2['str']
don2['coding_junction'] = don2['chr']+'_'+don2['min_exon_end'].astype('str')+'_'+don2['flank_exon_start'].astype('str')+'_'+don2['str']

In [327]:
start_time = time.time()
NMD_cols = ['NMD_junction']
can_cols = ['coding_junction']
           
acc1_summary = aaf.get_GTEX_summary_pooled(acc1,NMD_cols,can_cols)
acc2_summary = aaf.get_GTEX_summary_pooled(acc2,NMD_cols,can_cols)
don1_summary = aaf.get_GTEX_summary_pooled(don1,NMD_cols,can_cols)
don2_summary = aaf.get_GTEX_summary_pooled(don2,NMD_cols,can_cols)
time_took = time.time()-start_time

LQ or UQ are zero length: ABCA13 chr7_48494634_48494883_48506550_48506553_48506642_+
LQ or UQ are zero length: ABHD4 chr14_23070572_23070660_23072295_23072493_23072667_+
LQ or UQ are zero length: ABI2 chr2_204260379_204260503_204261508_204261607_204261690_+
no expression for: AC011479.1 chr19_38795015_38795130_38795205_38795264_38795326_+
done: 40 out of 1649
no expression for: ANKHD1-EIF4EBP3 chr5_139914947_139915123_139916923_139916974_139917174_+
done: 60 out of 1649
done: 80 out of 1649
no expression for: ASDURF chr2_190528627_190528680_190530102_190530148_190530177_+
no expression for: ATP5F1D chr19_1244096_1244184_1244314_1244318_1244439_+
no expression for: ATP5MC1 chr17_46971734_46971811_46972518_46972545_46972696_+
done: 120 out of 1649
done: 140 out of 1649
done: 160 out of 1649
done: 180 out of 1649
no expression for: CALHM4 chr6_116864928_116865035_116875354_116875448_116875514_+
done: 200 out of 1649
done: 240 out of 1649
done: 280 out of 1649
no expression for: CKLF-CMTM1

In [328]:
acc1_summary = pd.DataFrame(acc1_summary,columns = ['gene_name','AS event position','psi_L','psi_H',
                                            'log2FC_global','log2FC_local','zscore_global','zscore_local'])
acc2_summary = pd.DataFrame(acc2_summary,columns = ['gene_name','AS event position','psi_L','psi_H',
                                            'log2FC_global','log2FC_local','zscore_global','zscore_local'])
don1_summary = pd.DataFrame(don1_summary,columns = ['gene_name','AS event position','psi_L','psi_H',
                                            'log2FC_global','log2FC_local','zscore_global','zscore_local'])
don2_summary = pd.DataFrame(don2_summary,columns = ['gene_name','AS event position','psi_L','psi_H',
                                            'log2FC_global','log2FC_local','zscore_global','zscore_local'])
don_summary = pd.concat([don1_summary,don2_summary]).reset_index(drop=True)
acc_summary = pd.concat([acc1_summary,acc2_summary]).reset_index(drop=True)

In [332]:
don_summary.to_csv('./misc/hg19_coding_donor_summary_2.tsv', sep=str('\t'),header=True,index=None)
acc_summary.to_csv('./misc/hg19_coding_acceptor_summary_2.tsv', sep=str('\t'),header=True,index=None)

In [336]:
# protein-coding IR
hg19_intron_retention_coding = pd.read_csv('./misc/hg19_intron_retention_coding.tsv',delimiter="\t",
                                   index_col=None,header=0)
hg19_intron_retention_coding.rename(columns={'gene_name':'Target'},inplace=True)
hg19_intron_retention_coding['AS event position'] = hg19_intron_retention_coding['chr']+'_'+hg19_intron_retention_coding['upst_exon_start'].astype('str')+'_'+\
hg19_intron_retention_coding['upst_exon_end'].astype('str')+'_'+hg19_intron_retention_coding['downst_exon_start'].astype('str')+'_'+hg19_intron_retention_coding['downst_exon_end'].astype('str')+'_'+\
hg19_intron_retention_coding['str']
hg19_intron_retention_coding = hg19_intron_retention_coding.drop_duplicates(['Target','AS event position'])
hg19_intron_retention_coding['NMD_site_upst'] = hg19_intron_retention_coding['chr']+'_'+hg19_intron_retention_coding['upst_exon_end'].astype('str')+'_'+\
hg19_intron_retention_coding['str']
hg19_intron_retention_coding['NMD_site_downst'] = hg19_intron_retention_coding['chr']+'_'+hg19_intron_retention_coding['downst_exon_start'].astype('str')+'_'+\
hg19_intron_retention_coding['str']
hg19_intron_retention_coding['coding_junction'] = hg19_intron_retention_coding['chr']+'_'+hg19_intron_retention_coding['upst_exon_end'].astype('str')+'_'+\
hg19_intron_retention_coding['downst_exon_start'].astype('str')+'_'+hg19_intron_retention_coding['str']

In [ ]:
start_time = time.time()
NMD_cols = ['NMD_site_upst','NMD_site_downst']
can_cols = ['coding_junction']
           
ir_summary = aaf.get_GTEX_summary_pooled(hg19_intron_retention_coding,NMD_cols,can_cols)
time_took = time.time()-start_time

LQ or UQ are zero length: ABCA8 chr17_66917523_66917591_66917593_66917633_-
LQ or UQ are zero length: ABCA8 chr17_66925190_66925251_66925319_66925375_-
LQ or UQ are zero length: ABHD14B chr3_52005476_52005714_52005828_52005908_-
LQ or UQ are zero length: ACAN chr15_89398083_89398752_89398810_89402648_+
LQ or UQ are zero length: ACRV1 chr11_125547692_125547913_125548124_125548192_-
LQ or UQ are zero length: ACRV1 chr11_125547692_125547958_125548124_125548192_-
LQ or UQ are zero length: ADAR chr1_154573517_154574724_154574861_154575102_-
no expression for: AL391650.1 chr1_26498108_26498180_26498261_26498312_-
done: 40 out of 1006
no expression for: ATP5F1D chr19_1244314_1244439_1244736_1244824_+
done: 120 out of 1006
no expression for: CCNP chr19_40728115_40728379_40729059_40729210_-
no expression for: CFAP298 chr21_33973977_33974283_33975471_33975602_-
no expression for: CIAO2B chr16_66965968_66966104_66966175_66966203_-


In [342]:
ir_summary = pd.DataFrame(ir_summary,columns = ['gene_name','AS event position','psi_L','psi_H',
                                            'log2FC_global','log2FC_local','zscore_global','zscore_local'])

In [343]:
ir_summary.to_csv('./misc/hg19_coding_ir_summary.tsv', sep=str('\t'),header=True,index=None)

## Annotated USEs

In [349]:
# poison exons
nmd_sample = pd.read_csv('./novel_events_catalog/hg19_poison_exons.tsv',delimiter="\t",
                                   index_col=None,header=0)

In [350]:
nmd_sample['AS event position'] = nmd_sample['chr']+'_'+nmd_sample['upst_junction']+'_'+nmd_sample['downst_junction']+'_'+nmd_sample['str']
nmd_sample['coding_skip_junction'] = nmd_sample['chr']+'_'+nmd_sample['skip_junction']+'_'+nmd_sample['str']
nmd_sample['NMD_upst_junction'] = nmd_sample['chr']+'_'+nmd_sample['upst_junction']+'_'+nmd_sample['str']
nmd_sample['NMD_downst_junction'] = nmd_sample['chr']+'_'+nmd_sample['downst_junction']+'_'+nmd_sample['str']
nmd_sample.rename(columns={'gene_name':'Target'},inplace=True)
nmd_sample = nmd_sample.drop_duplicates(['Target','AS event position'])

In [352]:
NMD_cols = ['NMD_upst_junction','NMD_downst_junction']
can_cols = ['coding_skip_junction']

a = []
s = 0
while s<len(nmd_sample):
    e=s+1000
    temp_df = nmd_sample.iloc[s:min(e,len(nmd_sample))]
    temp_summary = aaf.get_GTEX_summary_pooled(temp_df,NMD_cols,can_cols)
    a = a+temp_summary
    print(str(s)+':'+str(e)+' done')
    s=s+1000

done: 0 out of 1000
LQ or UQ are zero length: ABCC5 chr3_183655848_183660221_183660303_183660515_-
no expression for: ABHD14A-ACY1 chr3_52012390_52018063_52018174_52019223_+
LQ or UQ are zero length: ABI2 chr2_204193354_204206920_204206953_204231600_+
no expression for: AC004832.3 chr22_30806668_30812223_30812392_30824449_+
no expression for: AC004922.1 chr7_98931040_98933076_98933107_98935804_+
no expression for: AC004997.1 chr22_30683549_30684695_30684765_30688762_-
no expression for: AC006059.2 chr3_42799843_42827520_42827654_42835649_-
done: 40 out of 1000
done: 80 out of 1000
no expression for: ATP5MF-PTCD1 chr7_99032891_99039013_99039110_99057709_-
no expression for: ATP5PO chr21_35276325_35279030_35279125_35279645_-
no expression for: BCLAF3 chrX_19935452_19941276_19941525_19947903_-
no expression for: BORCS8-MEF2B chr19_19261573_19291495_19291570_19293382_-
no expression for: C2orf92 chr2_98286297_98287654_98287769_98290919_+
done: 140 out of 1000
no expression for: CASTOR1 chr

In [353]:
a_df = pd.DataFrame(a,columns = ['gene_name','AS event position','psi_L','psi_H',
                                            'log2FC_global','log2FC_local','zscore_global','zscore_local'])

In [354]:
a_df.to_csv('./novel_events_catalog/hg19_poison_exons_summary.tsv', sep=str('\t'),header=True,index=None)

In [355]:
# essential exons
nmd_sample = pd.read_csv('./novel_events_catalog/hg19_essential_exons.tsv',delimiter="\t",
                                   index_col=None,header=0)

In [356]:
nmd_sample['AS event position'] = nmd_sample['chr']+'_'+nmd_sample['upst_junction']+'_'+nmd_sample['downst_junction']+'_'+nmd_sample['str']
nmd_sample['NMD_skip_junction'] = nmd_sample['chr']+'_'+nmd_sample['skip_junction']+'_'+nmd_sample['str']
nmd_sample['coding_upst_junction'] = nmd_sample['chr']+'_'+nmd_sample['upst_junction']+'_'+nmd_sample['str']
nmd_sample['coding_downst_junction'] = nmd_sample['chr']+'_'+nmd_sample['downst_junction']+'_'+nmd_sample['str']
nmd_sample.rename(columns={'gene_name':'Target'},inplace=True)
nmd_sample = nmd_sample.drop_duplicates(['Target','AS event position'])

In [357]:
NMD_cols = ['NMD_skip_junction']
can_cols = ['coding_upst_junction','coding_downst_junction']

a = []
s = 0
while s<len(nmd_sample):
    e=s+1000
    temp_df = nmd_sample.iloc[s:min(e,len(nmd_sample))]
    temp_summary = aaf.get_GTEX_summary_pooled(temp_df,NMD_cols,can_cols)
    a = a+temp_summary
    print(str(s)+':'+str(e)+' done')
    s=s+1000

done: 0 out of 1000
LQ or UQ are zero length: ABHD4 chr14_23072667_23072830_23072984_23075328_+
no expression for: ABRAXAS1 chr4_84391549_84393375_84393441_84397796_-
no expression for: AC004997.1 chr22_30683549_30688762_30688840_30689640_-
done: 20 out of 1000
no expression for: AFG1L chr6_108687536_108723200_108723258_108768417_+
done: 40 out of 1000
done: 60 out of 1000
no expression for: ATP5PO chr21_35279757_35281386_35281515_35284593_-
done: 80 out of 1000
no expression for: BUD23 chr7_73101230_73101329_73101425_73105246_+
no expression for: BUD23 chr7_73101425_73105246_73105342_73106926_+
done: 100 out of 1000
no expression for: CASTOR1 chr22_30683549_30684695_30684765_30685374_-
done: 120 out of 1000
no expression for: CCDC196 chr14_66957537_66957739_66957822_66958344_+
no expression for: CCN4 chr8_134203456_134225107_134225386_134237633_+
done: 140 out of 1000
no expression for: CFAP298 chr21_33976593_33979959_33980026_33982148_-
done: 160 out of 1000
no expression for: CFAP91

In [358]:
a_df = pd.DataFrame(a,columns = ['gene_name','AS event position','psi_L','psi_H',
                                            'log2FC_global','log2FC_local','zscore_global','zscore_local'])

In [359]:
a_df.to_csv('./novel_events_catalog/hg19_essential_exons_summary.tsv', sep=str('\t'),header=True,index=None)

In [361]:
# alt starts
nmd_sample = pd.read_csv('./novel_events_catalog/hg19_alt_starts.tsv',delimiter="\t",
                                   index_col=None,header=0)

In [364]:
nmd_sample['AS event position'] = nmd_sample['chr']+'_'+nmd_sample['flank_exon_start'].astype('str')+'_'+nmd_sample['flank_exon_end'].astype('str')+\
nmd_sample['exon_start'].astype('str')+'_'+nmd_sample['exon_start_coding'].astype('str')+'_'+nmd_sample['exon_end'].astype('str')+'_'+nmd_sample['str']

nmd_sample['NMD_junction'] = nmd_sample['chr']+'_'+nmd_sample['flank_exon_end'].astype('str')+'_'+nmd_sample['exon_start'].astype('str')+'_'+nmd_sample['str']
nmd_sample['coding_junction'] = nmd_sample['chr']+'_'+nmd_sample['flank_exon_end'].astype('str')+'_'+nmd_sample['exon_start_coding'].astype('str')+'_'+nmd_sample['str']
nmd_sample.rename(columns={'gene_name':'Target'},inplace=True)
nmd_sample = nmd_sample.drop_duplicates(['Target','AS event position'])

In [368]:
NMD_cols = ['NMD_junction']
can_cols = ['coding_junction']

a = []
s = 0
while s<len(nmd_sample):
    e=s+1000
    temp_df = nmd_sample.iloc[s:min(e,len(nmd_sample))]
    temp_summary = aaf.get_GTEX_summary_pooled(temp_df,NMD_cols,can_cols)
    a = a+temp_summary
    print(str(s)+':'+str(e)+' done')
    s=s+1000

LQ or UQ are zero length: AANAT chr17_74464754_7446499174465199_74465255_74465409_+
no expression for: AARS1 chr16_70288524_7028863770289623_70289631_70289739_-
no expression for: AARS1 chr16_70289631_7028973970291956_70291936_70292120_-
no expression for: AARS1 chr16_70316523_7031668770323433_70323324_70323458_-
LQ or UQ are zero length: ABCA7 chr19_1054779_10548771055082_1055096_1055350_+
LQ or UQ are zero length: ABCC12 chr16_48172139_4817228648173099_48173074_48173247_-
LQ or UQ are zero length: ABHD11 chr7_73151892_7315206573152663_73152658_73152793_-
LQ or UQ are zero length: ABHD5 chr3_43740768_4374085343743879_43743707_43744079_+
done: 20 out of 1000
no expression for: ADSS1 chr14_105205665_105205715105206104_105206087_105206153_+
done: 40 out of 1000
no expression for: AP000812.5 chr11_71816010_7181605871817102_71816982_71817256_+
done: 60 out of 1000
done: 80 out of 1000
no expression for: CCDC196 chr14_66957739_6695782266958325_66958344_66958403_+
no expression for: CCNQ chr

In [369]:
a_df = pd.DataFrame(a,columns = ['gene_name','AS event position','psi_L','psi_H',
                                            'log2FC_global','log2FC_local','zscore_global','zscore_local'])

In [370]:
a_df.to_csv('./novel_events_catalog/hg19_alt_starts_summary.tsv', sep=str('\t'),header=True,index=None)

In [371]:
# alt ends
nmd_sample = pd.read_csv('./novel_events_catalog/hg19_alt_ends.tsv',delimiter="\t",
                                   index_col=None,header=0)

In [372]:
nmd_sample['AS event position'] = nmd_sample['chr']+'_'+nmd_sample['exon_start'].astype('str')+'_'+nmd_sample['exon_end'].astype('str')+'_'+\
nmd_sample['exon_end_coding'].astype('str')+'_'+nmd_sample['flank_exon_start'].astype('str')+'_'+nmd_sample['flank_exon_end'].astype('str')+'_'+nmd_sample['str']

nmd_sample['NMD_junction'] = nmd_sample['chr']+'_'+nmd_sample['exon_end'].astype('str')+'_'+nmd_sample['flank_exon_start'].astype('str')+'_'+nmd_sample['str']
nmd_sample['coding_junction'] = nmd_sample['chr']+'_'+nmd_sample['exon_end_coding'].astype('str')+'_'+nmd_sample['flank_exon_start'].astype('str')+'_'+nmd_sample['str']
nmd_sample.rename(columns={'gene_name':'Target'},inplace=True)
nmd_sample = nmd_sample.drop_duplicates(['Target','AS event position'])

In [373]:
NMD_cols = ['NMD_junction']
can_cols = ['coding_junction']

a = []
s = 0
while s<len(nmd_sample):
    e=s+1000
    temp_df = nmd_sample.iloc[s:min(e,len(nmd_sample))]
    temp_summary = aaf.get_GTEX_summary_pooled(temp_df,NMD_cols,can_cols)
    a = a+temp_summary
    print(str(s)+':'+str(e)+' done')
    s=s+1000

no expression for: AARS1 chr16_70294947_70295053_70295060_70296249_70296427_-
no expression for: AARS1 chr16_70296249_70296387_70296427_70298861_70299005_-
no expression for: AARS1 chr16_70303521_70303737_70303666_70304099_70304243_-
LQ or UQ are zero length: ABCC8 chr11_17419886_17420010_17419988_17424208_17424300_-
LQ or UQ are zero length: ABCC8 chr11_17449836_17449944_17449952_17450112_17450217_-
done: 20 out of 1000
no expression for: AC118553.2 chr1_100459093_100459301_100459297_100464817_100464971_+
done: 40 out of 1000
done: 60 out of 1000
done: 80 out of 1000
no expression for: ATP5F1A chr18_43675019_43675113_43675097_43678138_43678262_-
done: 100 out of 1000
no expression for: BUD23 chr7_73098097_73098231_73098134_73100966_73101061_+
no expression for: CCDC197 chr14_94467491_94467665_94467669_94469569_94469700_+
no expression for: CCNP chr19_40729295_40729549_40729453_40730395_40730550_-
done: 160 out of 1000
no expression for: CEP20 chr16_15976806_15976905_15976877_15977865_

In [374]:
a_df = pd.DataFrame(a,columns = ['gene_name','AS event position','psi_L','psi_H',
                                            'log2FC_global','log2FC_local','zscore_global','zscore_local'])

In [375]:
a_df.to_csv('./novel_events_catalog/hg19_alt_ends_summary.tsv', sep=str('\t'),header=True,index=None)

In [376]:
# intron retention
nmd_sample = pd.read_csv('./novel_events_catalog/hg19_intron_retention.tsv',delimiter="\t",
                                   index_col=None,header=0)

In [377]:
nmd_sample['AS event position'] = nmd_sample['chr']+'_'+nmd_sample['exon_start'].astype('str')+'_'+nmd_sample['exon_end_coding'].astype('str')+'_'+\
nmd_sample['downst_exon_start_coding'].astype('str')+'_'+nmd_sample['exon_end'].astype('str')+'_'+nmd_sample['str']

nmd_sample['coding_junction'] = nmd_sample['chr']+'_'+nmd_sample['exon_end_coding'].astype('str')+'_'+nmd_sample['downst_exon_start_coding'].astype('str')+'_'+nmd_sample['str']
nmd_sample['NMD_site_upst'] = nmd_sample['chr']+'_'+nmd_sample['exon_end_coding'].astype('str')+'_'+nmd_sample['str']
nmd_sample['NMD_site_downst'] = nmd_sample['chr']+'_'+nmd_sample['downst_exon_start_coding'].astype('str')+'_'+nmd_sample['str']

nmd_sample.rename(columns={'gene_name':'Target'},inplace=True)
nmd_sample = nmd_sample.drop_duplicates(['Target','AS event position'])

In [378]:
NMD_cols = ['NMD_site_upst','NMD_site_downst']
can_cols = ['coding_junction']

a = []
s = 0
while s<len(nmd_sample):
    e=s+1000
    temp_df = nmd_sample.iloc[s:min(e,len(nmd_sample))]
    temp_summary = aaf.get_GTEX_summary_pooled(temp_df,NMD_cols,can_cols)
    a = a+temp_summary
    print(str(s)+':'+str(e)+' done')
    s=s+1000

done: 0 out of 93
LQ or UQ are zero length: APBB3 chr5_139941172_139941307_139941685_139941812_-
done: 20 out of 93
done: 40 out of 93
no expression for: PLPBP chr8_37623044_37623151_37623229_37623264_+
done: 60 out of 93
done: 80 out of 93
0:1000 done


In [379]:
a_df = pd.DataFrame(a,columns = ['gene_name','AS event position','psi_L','psi_H',
                                            'log2FC_global','log2FC_local','zscore_global','zscore_local'])

In [380]:
a_df.to_csv('./novel_events_catalog/hg19_intron_retention_summary.tsv', sep=str('\t'),header=True,index=None)

In [381]:
# intron detention
nmd_sample = pd.read_csv('./novel_events_catalog/hg19_intron_detention.tsv',delimiter="\t",
                                   index_col=None,header=0)

In [382]:
nmd_sample['AS event position'] = nmd_sample['chr']+'_'+nmd_sample['upst_exon_start'].astype('str')+'_'+nmd_sample['upst_exon_end'].astype('str')+'_'+\
nmd_sample['downst_exon_start'].astype('str')+'_'+nmd_sample['downst_exon_end'].astype('str')+'_'+nmd_sample['str']

nmd_sample['NMD_junction'] = nmd_sample['chr']+'_'+nmd_sample['upst_exon_end'].astype('str')+'_'+nmd_sample['downst_exon_start'].astype('str')+'_'+nmd_sample['str']
nmd_sample['coding_site_upst'] = nmd_sample['chr']+'_'+nmd_sample['upst_exon_end'].astype('str')+'_'+nmd_sample['str']
nmd_sample['coding_site_downst'] = nmd_sample['chr']+'_'+nmd_sample['downst_exon_start'].astype('str')+'_'+nmd_sample['str']

nmd_sample.rename(columns={'gene_name':'Target'},inplace=True)
nmd_sample = nmd_sample.drop_duplicates(['Target','AS event position'])

In [383]:
NMD_cols = ['NMD_junction']
can_cols = ['coding_site_upst','coding_site_downst']

a = []
s = 0
while s<len(nmd_sample):
    e=s+1000
    temp_df = nmd_sample.iloc[s:min(e,len(nmd_sample))]
    temp_summary = aaf.get_GTEX_summary_pooled(temp_df,NMD_cols,can_cols)
    a = a+temp_summary
    print(str(s)+':'+str(e)+' done')
    s=s+1000

LQ or UQ are zero length: ABHD4 chr14_23072295_23072401_23072506_23072667_+
LQ or UQ are zero length: ANGPTL4 chr19_8438589_8438839_8438900_8439254_+
LQ or UQ are zero length: ANKRD13B chr17_27940372_27940737_27941281_27941779_+
LQ or UQ are zero length: APOL3 chr22_36541521_36541547_36541625_36541647_-
done: 20 out of 148
done: 40 out of 148
done: 60 out of 148
done: 80 out of 148
done: 100 out of 148
no expression for: SHFL chr19_10199787_10199884_10199983_10200054_+
no expression for: TMEM250 chr9_139006435_139006796_139007302_139008870_-
done: 140 out of 148
0:1000 done


In [384]:
a_df = pd.DataFrame(a,columns = ['gene_name','AS event position','psi_L','psi_H',
                                            'log2FC_global','log2FC_local','zscore_global','zscore_local'])

In [385]:
a_df.to_csv('./novel_events_catalog/hg19_intron_detention_summary.tsv', sep=str('\t'),header=True,index=None)